# Evaluation Pipeline Using Markers as Features

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold

tqdm.pandas()

%matplotlib inline

%load_ext autoreload
%autoreload 2

Matplotlib created a temporary config/cache directory at /var/folders/kv/34jyv69d3lq7qnz8d_2l_s4w0000gq/T/matplotlib-t8zokufr because the default path (/Users/ibemmene/.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Fontconfig warning: ignoring UTF-8: not a valid region tag


In [2]:
SEED = 23
data_dir = Path("../Daten/data_to_use")


In [5]:
import sklearn
print(sklearn.__version__)

1.1.2


# Deep Learning models

In [12]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM
#keras wrapper thingy is deprecated
from scikeras.wrappers import KerasClassifier, KerasRegressor

Migration from Keras wrappers to scikeras: https://www.adriangb.com/scikeras/stable/migration.html

In [5]:
initializer = tf.keras.initializers.GlorotUniform(seed = SEED)



In [6]:
def create_rnn_model(first_layer = 8, second_layer = 4, shape = (22,1)):
    model = tf.keras.Sequential()

    model.add(SimpleRNN(first_layer, input_shape = shape, return_sequences=True, kernel_initializer = initializer))
    model.add(SimpleRNN(second_layer, return_sequences=False, kernel_initializer = initializer))
    model.add(Dense(1, activation = "sigmoid", kernel_initializer = initializer))
    
    model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics=[
        Precision(),
        Recall(),
    ])

    #model_rnn.summary()
    
    return model

rnn_model = KerasClassifier(model=create_rnn_model,first_layer = 8, second_layer = 4, shape = (22,1), epochs = 400, batch_size = 64, verbose=0)

In [7]:
def create_gru_model(first_layer = 16, second_layer = 8, shape = (22,1)):
    model = tf.keras.Sequential()

    model.add(GRU(16, input_shape = shape, return_sequences=True, kernel_initializer = initializer))
    model.add(GRU(8, return_sequences=False, kernel_initializer = initializer))
    model.add(Dense(1, activation = "sigmoid", kernel_initializer = initializer))
    model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics=[
        Precision(),
        Recall(),
    ])
    
    return model

gru_model = KerasClassifier(model=create_gru_model,first_layer = 16, second_layer = 8, shape = (22,1), epochs = 400, batch_size = 64,  verbose=0)

# Machine Learning Configuration

In [6]:
conf_ML = [
    ('Dummy', DummyClassifier(), {'Dummy__strategy' : ["stratified"], 'Dummy__random_state': [SEED]}),
    ('Tree', DecisionTreeClassifier(), {'Tree__max_depth': range(1,10), 'Tree__random_state': [SEED]}),
    ('Forest', RandomForestClassifier(), {'Forest__max_depth': range(1,20), 'Forest__random_state': [SEED]}),
    ('GradientBoost', GradientBoostingClassifier(), {'GradientBoost__max_depth': range(1,10), 'GradientBoost__random_state': [SEED]}),
    ('MLP', MLPClassifier(), {'MLP__hidden_layer_sizes': range(1,20), "MLP__random_state": [SEED]}),
    ('Logistic', LogisticRegression(), {'Logistic__C':np.logspace(0,4, 10) ,'Logistic__random_state': [SEED]}),
    #('Poly_SVM', SVC(), {'Poly_SVM__C' : [1.0, 2.5, 5.0, 7.5, 10.0], 'Poly_SVM__kernel' : ['poly'], 'Poly_SVM__degree' : range(1,3), 'Poly_SVM__random_state': [SEED]}),
    ('SVM', SVC(), {'SVM__C' : np.logspace(0,4, 20), 'SVM__gamma': np.logspace(-1,1, 10), 'SVM__kernel' : ['rbf'], 'SVM__random_state': [SEED]})
]

In [14]:
conf_DL = [
    ('RNN', rnn_model, {'RNN__first_layer' : [64, 32, 16], 'RNN__second_layer': [16,8,4]}),
    ('GRU', gru_model, {'GRU__first_layer' : [64,32, 16, 8], 'GRU__second_layer': [16,8,4], 'GRU__shape': []}),
]

NameError: name 'rnn_model' is not defined

# Data Handling Functions

In [7]:
features_ML = ["cv", "spread", "diffp", "rd", "rdnorm", "rdalt", "skew", "ks", "nbids"]

In [8]:
def prepare_data(name, features = features_ML, return_industry =False):
    
    datanames = [ "brazil_gaz", "japan", "brazil", "ticino", "US"]
    
    if name not in datanames + ["all"]:
        print("This is not a dataset ")
    
    if name == "all":
        data = []
        
        for ds in tqdm(datanames):
            df = pd.read_json(data_dir/f"{ds}_train.json")
            if "ID" in df.keys():
                df.rename(columns = {"ID": "project"}, inplace = True)
            if "Price" in df.keys():
                df.rename(columns = {"Price": "bid"}, inplace = True)
            if "complete" in df.keys(): #complete/incomplete is just a case in Swiss dataset, take care of it later
                df.rename(columns = {"complete": "cartel"}, inplace = True)
                
            df["project"] = df.project.apply(lambda s: f"{ds}/{s}" )
            
            if return_industry:
                df["industry"] = ds
            
            data.append(df[list(set(features + ["nbids"])) + ["bid", "project", "cartel"]]) #to make sure nbids is included as it will be needed later
        df = pd.concat(data)
        
    else:
        df = pd.read_json(data_dir/f"{name}_train.json")
    
        if "ID" in df.keys():
                        df.rename(columns = {"ID": "project"}, inplace = True)
        if "Price" in df.keys():
                        df.rename(columns = {"Price": "bid"}, inplace = True)
        df = df[list(set(features + ["nbids"])) + ["bid", "project", "cartel"]] #to make sure nbids is included as it will be needed later
    
    df_grouped = df.groupby(by="project").head(1).reset_index()
    
    df_grouped.drop(columns=["bid"], inplace = True)
    df_grouped = df_grouped.merge(df.groupby(by="project")["bid"].apply(list), left_on = "project", right_on = "project")
    
    df_grouped["bid"] = df_grouped.bid.apply(lambda x: (np.asarray(x)-min(np.asarray(x)))/np.std(np.asarray(x)))
    df_grouped["bid"] = df_grouped.bid.apply(lambda x: np.sort(x)[::-1])
    
    padding_length = max(np.asarray(df_grouped.nbids.tolist()))
    df_grouped["bid"] = df_grouped.bid.apply(lambda x: np.pad(x, (0,padding_length-len(x))))
    
    df_grouped.dropna(inplace = True)
    
    X_DL = np.array(df_grouped.bid.tolist())
    X_DL = X_DL.reshape(X_DL.shape[0], X_DL.shape[1], 1)

    X = df_grouped[features]
    y = df_grouped["cartel"]
    
    return X,y, X_DL

# Function Model Evaluation

In [9]:
def evaluate_models(X,y, configuration, SEED = SEED, print_full = False, LOO = False):
    model_scores = []
    models = []
    results = []
    full_results = []
    for name, algo, params in configuration:
        print(f'Trying {name} ...')
        # Extend model with normalisation step (here we use z-normalisation)
        #("scaler", StandardScaler()) # only needed in ML hyper parameter tuning
        pipeline = Pipeline([("scaler", StandardScaler()),(name, algo)])
        # Instantiate grid search object and pass params, number of folds and scoring-function
        if LOO:
            mycv = StratifiedKFold(n_splits = 10, random_state=SEED, shuffle = True) #not programmed yet
        else:
            mycv = StratifiedKFold(n_splits = 10, random_state=SEED, shuffle = True)
        classifier = GridSearchCV(pipeline, param_grid=params, cv= mycv, scoring=['f1', 'precision', 'recall', 'accuracy'], refit = 'f1', return_train_score = True)
        # Fit cross validation to our training data: within this function, all configurations are tried
        classifier.fit(X, y)
        print(f'  - Best hyperparameter: {classifier.best_params_}')
        print(f'  - Best F1 score: {classifier.best_score_:.3f}')
        if print_full:
            print(f'  - F1 Result for each parametrization in train:', [f'{x:.2f}' for x in classifier.cv_results_['mean_train_f1']])
            print(f'  - F1 Result for each parametrization in test:', [f'{x:.2f}' for x in classifier.cv_results_['mean_test_f1']])
            print(f'  - precision Result for each parametrization in train:', [f'{x:.2f}' for x in classifier.cv_results_['mean_train_precision']])
            print(f'  - precision for each parametrization in test:', [f'{x:.2f}' for x in classifier.cv_results_['mean_test_precision']])
            print(f'  - recall Result for each parametrization in train:', [f'{x:.2f}' for x in classifier.cv_results_['mean_train_recall']])
            print(f'  - recall Result for each parametrization in test:', [f'{x:.2f}' for x in classifier.cv_results_['mean_test_recall']])
            print(f'  - accuracy Result for each parametrization in train:', [f'{x:.2f}' for x in classifier.cv_results_['mean_train_accuracy']])
            print(f'  - accuracy Result for each parametrization in test:', [f'{x:.2f}' for x in classifier.cv_results_['mean_test_accuracy']])
        model_scores.append(classifier.best_score_)
        models.append(classifier.best_estimator_)
    
        index = classifier.best_index_
        df_results = pd.DataFrame({
            "algorithm": name,
            "train_f1": ["{:.3f}".format(classifier.cv_results_['mean_train_f1'][index])], "test_f1": ["{:.3f}".format(classifier.cv_results_['mean_test_f1'][index])],
            "train_precision": ["{:.3f}".format(classifier.cv_results_['mean_train_precision'][index])], "test_precision": ["{:.3f}".format(classifier.cv_results_['mean_test_precision'][index])],
            "train_recall": ["{:.3f}".format(classifier.cv_results_['mean_train_recall'][index])], "test_recall": ["{:.3f}".format(classifier.cv_results_['mean_test_recall'][index])],
            "train_accuracy": ["{:.3f}".format(classifier.cv_results_['mean_train_accuracy'][index])], "test_accuracy": ["{:.3f}".format(classifier.cv_results_['mean_test_accuracy'][index])],
            "params": [classifier.best_params_]
            })
        results.append(df_results)
    
        df_full_results = pd.DataFrame({
            "algorithm": name,
            "train_f1": classifier.cv_results_['mean_train_f1'], "test_f1": classifier.cv_results_['mean_test_f1'],
            "train_precision": classifier.cv_results_['mean_train_precision'], "test_precision": classifier.cv_results_['mean_test_precision'],
            "train_recall": classifier.cv_results_['mean_train_recall'], "test_recall": classifier.cv_results_['mean_test_recall'],
            "train_accuracy": classifier.cv_results_['mean_train_accuracy'], "test_recall": classifier.cv_results_['mean_test_accuracy'],
            "params": classifier.cv_results_["params"]
            })
        full_results.append(df_full_results)
    
    results = pd.concat(results)
    results = results.reset_index(drop = True)

    full_results = pd.concat(full_results)
    full_results = full_results.reset_index(drop = True)
    
    return results, full_results

# Classifiers on Features

# Try Optimization of DL

In [18]:
X,y, X_DL = prepare_data("japan", features = features_ML)


In [37]:

results, full_results = evaluate_models(X_DL, y, conf_DL, SEED = SEED)


Trying RNN ...
  - Best hyperparameter: {'RNN__first_layer': 32, 'RNN__second_layer': 8}
  - Best F1 score: 0.831
  - F1 Result for each parametrization in train: ['0.85', '0.83', '0.80', '0.81', '0.77', '0.72']
  - F1 Result for each parametrization in test: ['0.83', '0.82', '0.78', '0.81', '0.78', '0.75']
  - precision Result for each parametrization in train: ['0.83', '0.78', '0.74', '0.76', '0.68', '0.71']
  - precision for each parametrization in test: ['0.83', '0.78', '0.73', '0.77', '0.71', '0.73']
  - recall Result for each parametrization in train: ['0.86', '0.89', '0.89', '0.87', '0.89', '0.77']
  - recall Result for each parametrization in test: ['0.84', '0.88', '0.87', '0.85', '0.89', '0.78']


## Japan

In [19]:
results, full_results = evaluate_models(X, y, conf_ML, SEED = SEED)

Trying Dummy ...
  - Best hyperparameter: {'Dummy__random_state': 23, 'Dummy__strategy': 'stratified'}
  - Best F1 score: 0.582
Trying Tree ...
  - Best hyperparameter: {'Tree__max_depth': 2, 'Tree__random_state': 23}
  - Best F1 score: 0.936
Trying Forest ...
  - Best hyperparameter: {'Forest__max_depth': 5, 'Forest__random_state': 23}
  - Best F1 score: 0.950
Trying GradientBoost ...
  - Best hyperparameter: {'GradientBoost__max_depth': 6, 'GradientBoost__random_state': 23}
  - Best F1 score: 0.946
Trying MLP ...


/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum itera

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum itera

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum itera

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum itera

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum itera

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum itera

  - Best hyperparameter: {'MLP__hidden_layer_sizes': 4, 'MLP__random_state': 23}
  - Best F1 score: 0.924
Trying Logistic ...
  - Best hyperparameter: {'Logistic__C': 59.94842503189409, 'Logistic__random_state': 23}
  - Best F1 score: 0.918
Trying SVM ...
  - Best hyperparameter: {'SVM__C': 29.76351441631318, 'SVM__gamma': 0.46415888336127786, 'SVM__kernel': 'rbf', 'SVM__random_state': 23}
  - Best F1 score: 0.940


In [70]:
results, full_results = evaluate_models(X, y, conf_ML, SEED = SEED)

Trying Dummy ...
  - Best hyperparameter: {'Dummy__random_state': 23, 'Dummy__strategy': 'stratified'}
  - Best F1 score: 0.582
Trying Tree ...
  - Best hyperparameter: {'Tree__max_depth': 2, 'Tree__random_state': 23}
  - Best F1 score: 0.936
Trying Forest ...
  - Best hyperparameter: {'Forest__max_depth': 5, 'Forest__random_state': 23}
  - Best F1 score: 0.950
Trying GradientBoost ...
  - Best hyperparameter: {'GradientBoost__max_depth': 6, 'GradientBoost__random_state': 23}
  - Best F1 score: 0.946
Trying MLP ...


/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum itera

  - Best hyperparameter: {'MLP__hidden_layer_sizes': 1, 'MLP__random_state': 23}
  - Best F1 score: 0.914
Trying Logistic ...
  - Best hyperparameter: {'Logistic__C': 59.94842503189409, 'Logistic__random_state': 23}
  - Best F1 score: 0.918
Trying SVM ...
  - Best hyperparameter: {'SVM__C': 29.76351441631318, 'SVM__gamma': 0.46415888336127786, 'SVM__kernel': 'rbf', 'SVM__random_state': 23}
  - Best F1 score: 0.940


**Results with nbids included**

In [2]:
for i in range(1,3):
    print(i)

1
2


In [20]:
results

algorithm train_f1 test_f1 train_precision test_precision train_recall  \
0          Dummy    0.580   0.582           0.555          0.589        0.609   
1           Tree    0.937   0.936           0.910          0.912        0.966   
2         Forest    0.984   0.950           0.979          0.943        0.990   
3  GradientBoost    1.000   0.946           1.000          0.940        1.000   
4            MLP    0.924   0.924           0.924          0.920        0.923   
5       Logistic    0.921   0.918           0.926          0.923        0.916   
6            SVM    0.987   0.940           0.982          0.930        0.992   

  test_recall train_accuracy test_accuracy  \
0       0.575          0.488         0.520   
1       0.964          0.925         0.923   
2       0.960          0.981         0.942   
3       0.953          1.000         0.937   
4       0.928          0.911         0.911   
5       0.915          0.909         0.906   
6       0.951          0.985         0.930   

                                              params  
0  {'Dummy__random_state': 23, 'Dummy__strategy':...  
1   {'Tree__max_depth': 2, 'Tree__random_state': 23}  
2  {'Forest__max_depth': 5, 'Forest__random_state...  
3  {'GradientBoost__max_depth': 6, 'GradientBoost...  
4  {'MLP__hidden_layer_sizes': 4, 'MLP__random_st...  
5  {'Logistic__C': 59.94842503189409, 'Logistic__...  
6  {'SVM__C': 29.76351441631318, 'SVM__gamma': 0....

In [11]:
results

algorithm train_f1 test_f1 train_precision test_precision train_recall  \
0          Dummy    0.580   0.582           0.555          0.589        0.609   
1           Tree    0.937   0.936           0.910          0.912        0.966   
2         Forest    0.984   0.950           0.979          0.943        0.990   
3  GradientBoost    1.000   0.946           1.000          0.940        1.000   
4            MLP    0.924   0.924           0.924          0.920        0.923   
5       Logistic    0.921   0.918           0.926          0.923        0.916   
6            SVM    0.987   0.940           0.982          0.930        0.992   

  test_recall train_accuracy test_accuracy  \
0       0.575          0.488         0.520   
1       0.964          0.925         0.923   
2       0.960          0.981         0.942   
3       0.953          1.000         0.937   
4       0.928          0.911         0.911   
5       0.915          0.909         0.906   
6       0.951          0.985         0.930   

                                              params  
0  {'Dummy__random_state': 23, 'Dummy__strategy':...  
1   {'Tree__max_depth': 2, 'Tree__random_state': 23}  
2  {'Forest__max_depth': 5, 'Forest__random_state...  
3  {'GradientBoost__max_depth': 6, 'GradientBoost...  
4  {'MLP__hidden_layer_sizes': 4, 'MLP__random_st...  
5  {'Logistic__C': 59.94842503189409, 'Logistic__...  
6  {'SVM__C': 29.76351441631318, 'SVM__gamma': 0....

In [12]:
results

algorithm train_f1 test_f1 train_precision test_precision train_recall  \
0          Dummy    0.580   0.582           0.555          0.589        0.609   
1           Tree    0.937   0.936           0.910          0.912        0.966   
2         Forest    0.984   0.950           0.979          0.943        0.990   
3  GradientBoost    1.000   0.946           1.000          0.940        1.000   
4            MLP    0.919   0.914           0.924          0.918        0.914   
5       Logistic    0.921   0.918           0.926          0.923        0.916   
6            SVM    0.987   0.940           0.982          0.930        0.992   

  test_recall                                             params  
0       0.575  {'Dummy__random_state': 23, 'Dummy__strategy':...  
1       0.964   {'Tree__max_depth': 2, 'Tree__random_state': 23}  
2       0.960  {'Forest__max_depth': 5, 'Forest__random_state...  
3       0.953  {'GradientBoost__max_depth': 6, 'GradientBoost...  
4       0.913  {'MLP__hidden_layer_sizes': 1, 'MLP__random_st...  
5       0.915  {'Logistic__C': 59.94842503189409, 'Logistic__...  
6       0.951  {'SVM__C': 29.76351441631318, 'SVM__gamma': 0....

In [71]:
results


algorithm train_f1 test_f1 train_precision test_precision train_recall  \
0          Dummy    0.580   0.582           0.555          0.589        0.609   
1           Tree    0.937   0.936           0.910          0.912        0.966   
2         Forest    0.984   0.950           0.979          0.943        0.990   
3  GradientBoost    1.000   0.946           1.000          0.940        1.000   
4            MLP    0.919   0.914           0.924          0.918        0.914   
5       Logistic    0.921   0.918           0.926          0.923        0.916   
6            SVM    0.987   0.940           0.982          0.930        0.992   

  test_recall                                             params  
0       0.575  {'Dummy__random_state': 23, 'Dummy__strategy':...  
1       0.964   {'Tree__max_depth': 2, 'Tree__random_state': 23}  
2       0.960  {'Forest__max_depth': 5, 'Forest__random_state...  
3       0.953  {'GradientBoost__max_depth': 6, 'GradientBoost...  
4       0.913  {'MLP__hidden_layer_sizes': 1, 'MLP__random_st...  
5       0.915  {'Logistic__C': 59.94842503189409, 'Logistic__...  
6       0.951  {'SVM__C': 29.76351441631318, 'SVM__gamma': 0....

In [72]:
print(results.to_latex(index=False))  


\begin{tabular}{llllllll}
\toprule
    algorithm & train\_f1 & test\_f1 & train\_precision & test\_precision & train\_recall & test\_recall &                                             params \\
\midrule
        Dummy &    0.580 &   0.582 &           0.555 &          0.589 &        0.609 &       0.575 & \{'Dummy\_\_random\_state': 23, 'Dummy\_\_strategy': ... \\
         Tree &    0.937 &   0.936 &           0.910 &          0.912 &        0.966 &       0.964 &   \{'Tree\_\_max\_depth': 2, 'Tree\_\_random\_state': 23\} \\
       Forest &    0.984 &   0.950 &           0.979 &          0.943 &        0.990 &       0.960 & \{'Forest\_\_max\_depth': 5, 'Forest\_\_random\_state'... \\
GradientBoost &    1.000 &   0.946 &           1.000 &          0.940 &        1.000 &       0.953 & \{'GradientBoost\_\_max\_depth': 6, 'GradientBoost\_... \\
          MLP &    0.919 &   0.914 &           0.924 &          0.918 &        0.914 &       0.913 & \{'MLP\_\_hidden\_layer\_sizes': 1, 'MLP\_\_rand

/var/folders/kv/34jyv69d3lq7qnz8d_2l_s4w0000gq/T/ipykernel_67696/2548667327.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results.to_latex(index=False))


In [13]:
results.to_json("MLresults/japan_nbids_MLresults.json")
full_results.to_json("MLresults/japan_nbids_MLfullresults.json")

**Results without nbids included**

In [14]:
features_ML = ["cv", "spread", "diffp", "rd", "rdnorm", "rdalt", "skew", "ks"]
X,y, X_DL = prepare_data("japan", features = features_ML)
results, full_results = evaluate_models(X, y, conf_ML, SEED = SEED)


Trying Dummy ...
  - Best hyperparameter: {'Dummy__random_state': 23, 'Dummy__strategy': 'stratified'}
  - Best F1 score: 0.582
Trying Tree ...
  - Best hyperparameter: {'Tree__max_depth': 8, 'Tree__random_state': 23}
  - Best F1 score: 0.852
Trying Forest ...
  - Best hyperparameter: {'Forest__max_depth': 14, 'Forest__random_state': 23}
  - Best F1 score: 0.902
Trying GradientBoost ...
  - Best hyperparameter: {'GradientBoost__max_depth': 4, 'GradientBoost__random_state': 23}
  - Best F1 score: 0.911
Trying MLP ...


/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8

  - Best hyperparameter: {'MLP__hidden_layer_sizes': 2, 'MLP__random_state': 23}
  - Best F1 score: 0.803
Trying Logistic ...
  - Best hyperparameter: {'Logistic__C': 59.94842503189409, 'Logistic__random_state': 23}
  - Best F1 score: 0.832
Trying SVM ...
  - Best hyperparameter: {'SVM__C': 545.5594781168514, 'SVM__gamma': 0.16681005372000587, 'SVM__kernel': 'rbf', 'SVM__random_state': 23}
  - Best F1 score: 0.938


In [16]:
results.to_json("MLresults/japan_MLresults.json")
full_results.to_json("MLresults/japan_MLfullresults.json")

In [15]:
results

algorithm train_f1 test_f1 train_precision test_precision train_recall  \
0          Dummy    0.580   0.582           0.555          0.589        0.609   
1           Tree    0.972   0.852           0.974          0.843        0.969   
2         Forest    1.000   0.902           1.000          0.898        1.000   
3  GradientBoost    0.999   0.911           0.998          0.904        1.000   
4            MLP    0.808   0.803           0.732          0.724        0.901   
5       Logistic    0.836   0.832           0.783          0.779        0.898   
6            SVM    0.976   0.938           0.975          0.938        0.977   

  test_recall                                             params  
0       0.575  {'Dummy__random_state': 23, 'Dummy__strategy':...  
1       0.862   {'Tree__max_depth': 8, 'Tree__random_state': 23}  
2       0.907  {'Forest__max_depth': 14, 'Forest__random_stat...  
3       0.919  {'GradientBoost__max_depth': 4, 'GradientBoost...  
4       0.902  {'MLP__hidden_layer_sizes': 2, 'MLP__random_st...  
5       0.894  {'Logistic__C': 59.94842503189409, 'Logistic__...  
6       0.940  {'SVM__C': 545.5594781168514, 'SVM__gamma': 0....

In [26]:
results

algorithm train_f1 test_f1 train_precision test_precision train_recall  \
0          Dummy    0.580   0.582           0.555          0.589        0.609   
1           Tree    0.972   0.852           0.974          0.843        0.969   
2         Forest    1.000   0.902           1.000          0.898        1.000   
3  GradientBoost    0.999   0.911           0.998          0.904        1.000   
4       Logistic    0.821   0.816           0.745          0.742        0.915   
5            SVM    0.978   0.939           0.977          0.938        0.979   

  test_recall                                             params  
0       0.575  {'Dummy__random_state': 23, 'Dummy__strategy':...  
1       0.862   {'Tree__max_depth': 8, 'Tree__random_state': 23}  
2       0.907  {'Forest__max_depth': 14, 'Forest__random_stat...  
3       0.919  {'GradientBoost__max_depth': 4, 'GradientBoost...  
4       0.909                     {'Logistic__random_state': 23}  
5       0.943  {'SVM__C': 774.263682681127, 'SVM__gamma': 0.1...

In [27]:
print(results.to_latex(index=False))  

\begin{tabular}{llllllll}
\toprule
    algorithm & train\_f1 & test\_f1 & train\_precision & test\_precision & train\_recall & test\_recall &                                             params \\
\midrule
        Dummy &    0.580 &   0.582 &           0.555 &          0.589 &        0.609 &       0.575 & \{'Dummy\_\_random\_state': 23, 'Dummy\_\_strategy': ... \\
         Tree &    0.972 &   0.852 &           0.974 &          0.843 &        0.969 &       0.862 &   \{'Tree\_\_max\_depth': 8, 'Tree\_\_random\_state': 23\} \\
       Forest &    1.000 &   0.902 &           1.000 &          0.898 &        1.000 &       0.907 & \{'Forest\_\_max\_depth': 14, 'Forest\_\_random\_state... \\
GradientBoost &    0.999 &   0.911 &           0.998 &          0.904 &        1.000 &       0.919 & \{'GradientBoost\_\_max\_depth': 4, 'GradientBoost\_... \\
     Logistic &    0.821 &   0.816 &           0.745 &          0.742 &        0.915 &       0.909 &                     \{'Logistic\_\_random\_state

/var/folders/kv/34jyv69d3lq7qnz8d_2l_s4w0000gq/T/ipykernel_67696/3872705694.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results.to_latex(index=False))


## Brasil

Run this on 16th of January 2024 again for new paper again

In [10]:
conf_ML = [
    ('Dummy', DummyClassifier(), {'Dummy__strategy' : ["stratified"], 'Dummy__random_state': [SEED]}),
    ('Tree', DecisionTreeClassifier(), {'Tree__max_depth': range(1,10), 'Tree__random_state': [SEED]}),
    ('Forest', RandomForestClassifier(), {'Forest__max_depth': range(1,20), 'Forest__random_state': [SEED]}),
    ('GradientBoost', GradientBoostingClassifier(), {'GradientBoost__max_depth': range(1,10), 'GradientBoost__random_state': [SEED]}),
    ('MLP', MLPClassifier(), {'MLP__hidden_layer_sizes': range(1,20), "MLP__random_state": [SEED]}),
    ('Logistic', LogisticRegression(), {'Logistic__C':np.logspace(0,4, 10) ,'Logistic__random_state': [SEED]}),
    ('Poly_SVM', SVC(), {'Poly_SVM__C' : np.logspace(0,4, 20), 'Poly_SVM__kernel' : ['poly'], 'Poly_SVM__degree' : range(0,4), 'Poly_SVM__random_state': [SEED]}),
    ('SVM', SVC(), {'SVM__C' : np.logspace(0,4, 20), 'SVM__gamma': np.logspace(-1,1, 10), 'SVM__kernel' : ['rbf'], 'SVM__random_state': [SEED]})
]

In [11]:
features_ML = ["cv", "spread", "diffp", "rd", "rdnorm", "rdalt", "skew", "ks", "nbids"]
X,y, X_DL = prepare_data("brazil_gaz", features = features_ML)
results, full_results = evaluate_models(X, y, conf_ML, SEED = SEED)

Trying Dummy ...
  - Best hyperparameter: {'Dummy__random_state': 23, 'Dummy__strategy': 'stratified'}
  - Best F1 score: 0.470
Trying Tree ...
  - Best hyperparameter: {'Tree__max_depth': 8, 'Tree__random_state': 23}
  - Best F1 score: 0.933
Trying Forest ...
  - Best hyperparameter: {'Forest__max_depth': 16, 'Forest__random_state': 23}
  - Best F1 score: 0.936
Trying GradientBoost ...
  - Best hyperparameter: {'GradientBoost__max_depth': 5, 'GradientBoost__random_state': 23}
  - Best F1 score: 0.963
Trying MLP ...


/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum itera

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum itera

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum itera

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum itera

...

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

  - Best hyperparameter: {'Poly_SVM__C': 3792.690190732246, 'Poly_SVM__degree': 2, 'Poly_SVM__kernel': 'poly', 'Poly_SVM__random_state': 23}
  - Best F1 score: 0.846
Trying SVM ...
  - Best hyperparameter: {'SVM__C': 29.76351441631318, 'SVM__gamma': 0.774263682681127, 'SVM__kernel': 'rbf', 'SVM__random_state': 23}
  - Best F1 score: 0.881


**Additional Result original Evaulation**

In [29]:
results

algorithm train_f1 test_f1 train_precision test_precision train_recall  \
0       MLP    0.852   0.843           0.898          0.887        0.810   

  test_recall                                             params  
0       0.805  {'MLP__hidden_layer_sizes': 17, 'MLP__random_s...

In [30]:
print(results.to_latex(index=False))  

\begin{tabular}{llllllll}
\toprule
algorithm & train\_f1 & test\_f1 & train\_precision & test\_precision & train\_recall & test\_recall &                                             params \\
\midrule
      MLP &    0.852 &   0.843 &           0.898 &          0.887 &        0.810 &       0.805 & \{'MLP\_\_hidden\_layer\_sizes': 17, 'MLP\_\_random\_st... \\
\bottomrule
\end{tabular}



/var/folders/kv/34jyv69d3lq7qnz8d_2l_s4w0000gq/T/ipykernel_35134/3872705694.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results.to_latex(index=False))


**Results with nbids**

In [15]:
results.to_json("MLresults/brazil_gaz_nbids_MLresults_reeval.json")
full_results.to_json("MLresults/brazil_gaz_nbids_MLfullresults_reeval.json")

**Results Reevaluation 16. Jan 2024**

In [13]:
results

algorithm train_f1 test_f1 train_precision test_precision train_recall  \
0          Dummy    0.450   0.470           0.470          0.482        0.431   
1           Tree    0.972   0.933           0.966          0.932        0.978   
2         Forest    1.000   0.936           1.000          0.957        1.000   
3  GradientBoost    1.000   0.963           1.000          0.970        1.000   
4            MLP    0.852   0.843           0.898          0.887        0.810   
5       Logistic    0.805   0.797           0.834          0.823        0.778   
6       Poly_SVM    0.861   0.846           0.910          0.897        0.817   
7            SVM    0.972   0.881           0.982          0.884        0.961   

  test_recall train_accuracy test_accuracy  \
0       0.459          0.493         0.503   
1       0.935          0.973         0.935   
2       0.916          1.000         0.939   
3       0.957          1.000         0.965   
4       0.805          0.865         0.856   
5       0.774          0.819         0.811   
6       0.802          0.873         0.860   
7       0.878          0.973         0.886   

                                              params  
0  {'Dummy__random_state': 23, 'Dummy__strategy':...  
1   {'Tree__max_depth': 8, 'Tree__random_state': 23}  
2  {'Forest__max_depth': 16, 'Forest__random_stat...  
3  {'GradientBoost__max_depth': 5, 'GradientBoost...  
4  {'MLP__hidden_layer_sizes': 17, 'MLP__random_s...  
5  {'Logistic__C': 21.544346900318832, 'Logistic_...  
6  {'Poly_SVM__C': 3792.690190732246, 'Poly_SVM__...  
7  {'SVM__C': 29.76351441631318, 'SVM__gamma': 0....

In [14]:
print(results.to_latex(index=False)) 

\begin{tabular}{llllllllll}
\toprule
    algorithm & train\_f1 & test\_f1 & train\_precision & test\_precision & train\_recall & test\_recall & train\_accuracy & test\_accuracy &                                             params \\
\midrule
        Dummy &    0.450 &   0.470 &           0.470 &          0.482 &        0.431 &       0.459 &          0.493 &         0.503 & \{'Dummy\_\_random\_state': 23, 'Dummy\_\_strategy': ... \\
         Tree &    0.972 &   0.933 &           0.966 &          0.932 &        0.978 &       0.935 &          0.973 &         0.935 &   \{'Tree\_\_max\_depth': 8, 'Tree\_\_random\_state': 23\} \\
       Forest &    1.000 &   0.936 &           1.000 &          0.957 &        1.000 &       0.916 &          1.000 &         0.939 & \{'Forest\_\_max\_depth': 16, 'Forest\_\_random\_state... \\
GradientBoost &    1.000 &   0.963 &           1.000 &          0.970 &        1.000 &       0.957 &          1.000 &         0.965 & \{'GradientBoost\_\_max\_depth': 5, 'Gr

/var/folders/kv/34jyv69d3lq7qnz8d_2l_s4w0000gq/T/ipykernel_1359/516983427.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results.to_latex(index=False))


**Original Results**

In [74]:
results

algorithm train_f1 test_f1 train_precision test_precision train_recall  \
0          Dummy    0.450   0.470           0.470          0.482        0.431   
1           Tree    0.972   0.933           0.966          0.932        0.978   
2         Forest    1.000   0.936           1.000          0.957        1.000   
3  GradientBoost    1.000   0.963           1.000          0.970        1.000   
4            MLP    0.807   0.801           0.864          0.853        0.758   
5       Logistic    0.805   0.797           0.834          0.823        0.778   
6            SVM    0.972   0.881           0.982          0.884        0.961   

  test_recall                                             params  
0       0.459  {'Dummy__random_state': 23, 'Dummy__strategy':...  
1       0.935   {'Tree__max_depth': 8, 'Tree__random_state': 23}  
2       0.916  {'Forest__max_depth': 16, 'Forest__random_stat...  
3       0.957  {'GradientBoost__max_depth': 5, 'GradientBoost...  
4       0.755  {'MLP__hidden_layer_sizes': 2, 'MLP__random_st...  
5       0.774  {'Logistic__C': 21.544346900318832, 'Logistic_...  
6       0.878  {'SVM__C': 29.76351441631318, 'SVM__gamma': 0....

In [75]:
print(results.to_latex(index=False))  

\begin{tabular}{llllllll}
\toprule
    algorithm & train\_f1 & test\_f1 & train\_precision & test\_precision & train\_recall & test\_recall &                                             params \\
\midrule
        Dummy &    0.450 &   0.470 &           0.470 &          0.482 &        0.431 &       0.459 & \{'Dummy\_\_random\_state': 23, 'Dummy\_\_strategy': ... \\
         Tree &    0.972 &   0.933 &           0.966 &          0.932 &        0.978 &       0.935 &   \{'Tree\_\_max\_depth': 8, 'Tree\_\_random\_state': 23\} \\
       Forest &    1.000 &   0.936 &           1.000 &          0.957 &        1.000 &       0.916 & \{'Forest\_\_max\_depth': 16, 'Forest\_\_random\_state... \\
GradientBoost &    1.000 &   0.963 &           1.000 &          0.970 &        1.000 &       0.957 & \{'GradientBoost\_\_max\_depth': 5, 'GradientBoost\_... \\
          MLP &    0.807 &   0.801 &           0.864 &          0.853 &        0.758 &       0.755 & \{'MLP\_\_hidden\_layer\_sizes': 2, 'MLP\_\_rand

/var/folders/kv/34jyv69d3lq7qnz8d_2l_s4w0000gq/T/ipykernel_67696/3872705694.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results.to_latex(index=False))


In [31]:
features_ML = ["cv", "spread", "diffp", "rd", "rdnorm", "rdalt", "skew", "ks"]
X,y, X_DL = prepare_data("brazil_gaz", features = features_ML)
results, full_results = evaluate_models(X, y, conf_ML, SEED = SEED)

Trying Dummy ...
  - Best hyperparameter: {'Dummy__random_state': 23, 'Dummy__strategy': 'stratified'}
  - Best F1 score: 0.470
Trying Tree ...
  - Best hyperparameter: {'Tree__max_depth': 7, 'Tree__random_state': 23}
  - Best F1 score: 0.792
Trying Forest ...
  - Best hyperparameter: {'Forest__max_depth': 12, 'Forest__random_state': 23}
  - Best F1 score: 0.867
Trying GradientBoost ...
  - Best hyperparameter: {'GradientBoost__max_depth': 5, 'GradientBoost__random_state': 23}
  - Best F1 score: 0.871
Trying Logistic ...
  - Best hyperparameter: {'Logistic__random_state': 23}
  - Best F1 score: 0.766
Trying SVM ...
  - Best hyperparameter: {'SVM__C': 215.44346900318845, 'SVM__gamma': 0.46415888336127786, 'SVM__kernel': 'rbf', 'SVM__random_state': 23}
  - Best F1 score: 0.861


**Results without nbids**

In [32]:
results

algorithm train_f1 test_f1 train_precision test_precision train_recall  \
0          Dummy    0.450   0.470           0.470          0.482        0.431   
1           Tree    0.862   0.792           0.872          0.802        0.855   
2         Forest    0.989   0.867           0.999          0.887        0.979   
3  GradientBoost    0.992   0.871           0.993          0.889        0.991   
4       Logistic    0.771   0.766           0.811          0.804        0.735   
5            SVM    0.950   0.861           0.955          0.863        0.944   

  test_recall                                             params  
0       0.459  {'Dummy__random_state': 23, 'Dummy__strategy':...  
1       0.789   {'Tree__max_depth': 7, 'Tree__random_state': 23}  
2       0.848  {'Forest__max_depth': 12, 'Forest__random_stat...  
3       0.855  {'GradientBoost__max_depth': 5, 'GradientBoost...  
4       0.734                     {'Logistic__random_state': 23}  
5       0.861  {'SVM__C': 215.44346900318845, 'SVM__gamma': 0...

In [33]:
print(results.to_latex(index=False))  

\begin{tabular}{llllllll}
\toprule
    algorithm & train\_f1 & test\_f1 & train\_precision & test\_precision & train\_recall & test\_recall &                                             params \\
\midrule
        Dummy &    0.450 &   0.470 &           0.470 &          0.482 &        0.431 &       0.459 & \{'Dummy\_\_random\_state': 23, 'Dummy\_\_strategy': ... \\
         Tree &    0.862 &   0.792 &           0.872 &          0.802 &        0.855 &       0.789 &   \{'Tree\_\_max\_depth': 7, 'Tree\_\_random\_state': 23\} \\
       Forest &    0.989 &   0.867 &           0.999 &          0.887 &        0.979 &       0.848 & \{'Forest\_\_max\_depth': 12, 'Forest\_\_random\_state... \\
GradientBoost &    0.992 &   0.871 &           0.993 &          0.889 &        0.991 &       0.855 & \{'GradientBoost\_\_max\_depth': 5, 'GradientBoost\_... \\
     Logistic &    0.771 &   0.766 &           0.811 &          0.804 &        0.735 &       0.734 &                     \{'Logistic\_\_random\_state

/var/folders/kv/34jyv69d3lq7qnz8d_2l_s4w0000gq/T/ipykernel_67696/3872705694.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results.to_latex(index=False))


# Use all Dataset Together including additional features


In [ ]:
features_ML = ["cv", "spread", "diffp", "rd", "rdnorm", "rdalt", "skew", "ks", "nbids"]
X,y, X_DL = prepare_data("all", features = features_ML)
results, full_results = evaluate_models(X, y, conf_ML, SEED = SEED)

  0%|          | 0/5 [00:00<?, ?it/s]

Trying MLP ...


/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum itera

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum itera

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum itera

...

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

**Results with nbids included as feature**

In [21]:
results.to_json("MLresults/all_nbids_MLresults.json")
full_results.to_json("MLresults/all_nbids_MLfullresults.json")

In [77]:
results

algorithm train_f1 test_f1 train_precision test_precision train_recall  \
0          Dummy    0.468   0.472           0.475          0.487        0.461   
1           Tree    0.902   0.826           0.960          0.882        0.850   
2         Forest    0.997   0.852           1.000          0.886        0.994   
3  GradientBoost    0.976   0.863           0.990          0.898        0.961   
4            MLP    0.719   0.715           0.750          0.742        0.691   
5       Logistic    0.614   0.613           0.654          0.654        0.579   
6            SVM    0.913   0.810           0.963          0.845        0.868   

  test_recall                                             params  
0       0.457  {'Dummy__random_state': 23, 'Dummy__strategy':...  
1       0.777   {'Tree__max_depth': 9, 'Tree__random_state': 23}  
2       0.821  {'Forest__max_depth': 19, 'Forest__random_stat...  
3       0.832  {'GradientBoost__max_depth': 6, 'GradientBoost...  
4       0.690  {'MLP__hidden_layer_sizes': 2, 'MLP__random_st...  
5       0.578  {'Logistic__C': 7.742636826811269, 'Logistic__...  
6       0.779  {'SVM__C': 48.32930238571752, 'SVM__gamma': 1....

In [78]:
print(results.to_latex(index=False))  

\begin{tabular}{llllllll}
\toprule
    algorithm & train\_f1 & test\_f1 & train\_precision & test\_precision & train\_recall & test\_recall &                                             params \\
\midrule
        Dummy &    0.468 &   0.472 &           0.475 &          0.487 &        0.461 &       0.457 & \{'Dummy\_\_random\_state': 23, 'Dummy\_\_strategy': ... \\
         Tree &    0.902 &   0.826 &           0.960 &          0.882 &        0.850 &       0.777 &   \{'Tree\_\_max\_depth': 9, 'Tree\_\_random\_state': 23\} \\
       Forest &    0.997 &   0.852 &           1.000 &          0.886 &        0.994 &       0.821 & \{'Forest\_\_max\_depth': 19, 'Forest\_\_random\_state... \\
GradientBoost &    0.976 &   0.863 &           0.990 &          0.898 &        0.961 &       0.832 & \{'GradientBoost\_\_max\_depth': 6, 'GradientBoost\_... \\
          MLP &    0.719 &   0.715 &           0.750 &          0.742 &        0.691 &       0.690 & \{'MLP\_\_hidden\_layer\_sizes': 2, 'MLP\_\_rand

/var/folders/kv/34jyv69d3lq7qnz8d_2l_s4w0000gq/T/ipykernel_67696/3872705694.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results.to_latex(index=False))


!!! saw that for SVM the chosen paramtere is at the limit of the grid, perhpas need to retry !!!

In [37]:
features_ML = ["cv", "spread", "diffp", "rd", "rdnorm", "rdalt", "skew", "ks"]
X,y, X_DL = prepare_data("all", features = features_ML)
results, full_results = evaluate_models(X, y, conf_ML, SEED = SEED)

  0%|          | 0/5 [00:00<?, ?it/s]

Trying Dummy ...
  - Best hyperparameter: {'Dummy__random_state': 23, 'Dummy__strategy': 'stratified'}
  - Best F1 score: 0.472
Trying Tree ...
  - Best hyperparameter: {'Tree__max_depth': 9, 'Tree__random_state': 23}
  - Best F1 score: 0.700
Trying Forest ...
  - Best hyperparameter: {'Forest__max_depth': 16, 'Forest__random_state': 23}
  - Best F1 score: 0.798
Trying GradientBoost ...
  - Best hyperparameter: {'GradientBoost__max_depth': 7, 'GradientBoost__random_state': 23}
  - Best F1 score: 0.788
Trying Logistic ...
  - Best hyperparameter: {'Logistic__random_state': 23}
  - Best F1 score: 0.610
Trying SVM ...
  - Best hyperparameter: {'SVM__C': 464.15888336127773, 'SVM__gamma': 0.774263682681127, 'SVM__kernel': 'rbf', 'SVM__random_state': 23}
  - Best F1 score: 0.776


**Results without nbids included as feature**

In [38]:
results

algorithm train_f1 test_f1 train_precision test_precision train_recall  \
0          Dummy    0.468   0.472           0.475          0.487        0.461   
1           Tree    0.792   0.700           0.900          0.796        0.709   
2         Forest    0.988   0.798           1.000          0.841        0.977   
3  GradientBoost    0.981   0.788           0.996          0.826        0.967   
4       Logistic    0.620   0.610           0.637          0.629        0.605   
5            SVM    0.888   0.776           0.946          0.821        0.838   

  test_recall                                             params  
0       0.457  {'Dummy__random_state': 23, 'Dummy__strategy':...  
1       0.626   {'Tree__max_depth': 9, 'Tree__random_state': 23}  
2       0.760  {'Forest__max_depth': 16, 'Forest__random_stat...  
3       0.753  {'GradientBoost__max_depth': 7, 'GradientBoost...  
4       0.594                     {'Logistic__random_state': 23}  
5       0.736  {'SVM__C': 464.15888336127773, 'SVM__gamma': 0...

**TODO:** These results are not identical with what we had previously needs to be investigated.

In [39]:
print(results.to_latex(index=False))  

\begin{tabular}{llllllll}
\toprule
    algorithm & train\_f1 & test\_f1 & train\_precision & test\_precision & train\_recall & test\_recall &                                             params \\
\midrule
        Dummy &    0.468 &   0.472 &           0.475 &          0.487 &        0.461 &       0.457 & \{'Dummy\_\_random\_state': 23, 'Dummy\_\_strategy': ... \\
         Tree &    0.792 &   0.700 &           0.900 &          0.796 &        0.709 &       0.626 &   \{'Tree\_\_max\_depth': 9, 'Tree\_\_random\_state': 23\} \\
       Forest &    0.988 &   0.798 &           1.000 &          0.841 &        0.977 &       0.760 & \{'Forest\_\_max\_depth': 16, 'Forest\_\_random\_state... \\
GradientBoost &    0.981 &   0.788 &           0.996 &          0.826 &        0.967 &       0.753 & \{'GradientBoost\_\_max\_depth': 7, 'GradientBoost\_... \\
     Logistic &    0.620 &   0.610 &           0.637 &          0.629 &        0.605 &       0.594 &                     \{'Logistic\_\_random\_state

/var/folders/kv/34jyv69d3lq7qnz8d_2l_s4w0000gq/T/ipykernel_67696/3872705694.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results.to_latex(index=False))


**Let's see what to do with this**

In [52]:
df_data.corr()

project    cartel        cv    spread     diffp  \
project                  1.000000 -0.525889  0.044125 -0.033462  0.076035   
cartel                  -0.525889  1.000000 -0.149849 -0.080118 -0.120781   
cv                       0.044125 -0.149849  1.000000  0.922287  0.352781   
spread                  -0.033462 -0.080118  0.922287  1.000000  0.260848   
diffp                    0.076035 -0.120781  0.352781  0.260848  1.000000   
rd                       0.055356  0.037921 -0.006378 -0.029533  0.272715   
rdnorm                  -0.114093  0.145861 -0.150096 -0.054245  0.067016   
rdalt                   -0.063897  0.123668 -0.105958 -0.047708  0.148962   
skew                     0.118148 -0.130201  0.184599  0.142449 -0.234556   
ks                      -0.246435  0.235085 -0.212096 -0.025249 -0.129041   
industry_construction   -0.378364  0.165817  0.062060  0.061588  0.087506   
industry_gasoline       -0.042383 -0.001037 -0.205896 -0.048083 -0.318045   
industry_milk            0.487353 -0.189678  0.179499 -0.012313  0.286895   
pricing_first-price_SBA  0.042383  0.001037  0.205896  0.048083  0.318045   
pricing_pricing         -0.042383 -0.001037 -0.205896 -0.048083 -0.318045   

                               rd    rdnorm     rdalt      skew        ks  \
project                  0.055356 -0.114093 -0.063897  0.118148 -0.246435   
cartel                   0.037921  0.145861  0.123668 -0.130201  0.235085   
cv                      -0.006378 -0.150096 -0.105958  0.184599 -0.212096   
spread                  -0.029533 -0.054245 -0.047708  0.142449 -0.025249   
diffp                    0.272715  0.067016  0.148962 -0.234556 -0.129041   
rd                       1.000000  0.109073  0.463497 -0.274677 -0.000938   
rdnorm                   0.109073  1.000000  0.837017 -0.323475  0.542648   
rdalt                    0.463497  0.837017  1.000000 -0.390746  0.447196   
skew                    -0.274677 -0.323475 -0.390746  1.000000 -0.190782   
ks                      -0.000938  0.542648  0.447196 -0.190782  1.000000   
industry_construction    0.037998 -0.080307 -0.028878 -0.193198 -0.123009   
industry_gasoline       -0.157864  0.265614  0.125121  0.077167  0.515870   
industry_milk            0.148661 -0.231277 -0.119254  0.128414 -0.487137   
pricing_first-price_SBA  0.157864 -0.265614 -0.125121 -0.077167 -0.515870   
pricing_pricing         -0.157864  0.265614  0.125121  0.077167  0.515870   

                         industry_construction  industry_gasoline  \
project                              -0.378364          -0.042383   
cartel                                0.165817          -0.001037   
cv                                    0.062060          -0.205896   
spread                                0.061588          -0.048083   
diffp                                 0.087506          -0.318045   
rd                                    0.037998          -0.157864   
rdnorm                               -0.080307           0.265614   
rdalt                                -0.028878           0.125121   
skew                                 -0.193198           0.077167   
ks                                   -0.123009           0.515870   
industry_construction                 1.000000          -0.645381   
industry_gasoline                    -0.645381           1.000000   
industry_milk                        -0.364876          -0.475714   
pricing_first-price_SBA               0.645381          -1.000000   
pricing_pricing                      -0.645381           1.000000   

                         industry_milk  pricing_first-price_SBA  \
project                       0.487353                 0.042383   
cartel                       -0.189678                 0.001037   
cv                            0.179499                 0.205896   
spread                       -0.012313                 0.048083   
diffp                         0.286895                 0.318045   
rd                            0.148661               

In [56]:
full_results

algorithm  train_f1   test_f1  train_precision  test_precision  \
0           Dummy  0.467846  0.471707         0.474867        0.487324   
1            Tree  0.614552  0.612656         0.809077        0.807372   
2            Tree  0.641224  0.635523         0.811720        0.806974   
3            Tree  0.670465  0.647707         0.781542        0.750211   
4            Tree  0.686705  0.663311         0.818339        0.792048   
5            Tree  0.708498  0.672655         0.827070        0.798349   
6            Tree  0.731970  0.691901         0.847858        0.814526   
7            Tree  0.753462  0.685709         0.881399        0.817699   
8            Tree  0.774971  0.699267         0.893634        0.810391   
9            Tree  0.799970  0.704009         0.912745        0.804238   
10           Tree  0.834021  0.727329         0.914505        0.798499   
11           Tree  0.860591  0.733304         0.934882        0.795237   
12           Tree  0.887347  0.731720         0.949368        0.785041   
13           Tree  0.910098  0.741265         0.967002        0.789219   
14           Tree  0.927858  0.747676         0.979131        0.788209   
15           Tree  0.941522  0.741275         0.985867        0.773446   
16           Tree  0.953884  0.740254         0.988611        0.766874   
17           Tree  0.963480  0.741419         0.991219        0.762463   
18           Tree  0.971585  0.748588         0.993053        0.763514   
19           Tree  0.977878  0.746458         0.994084        0.758961   
20         Forest  0.613739  0.611928         0.807826        0.806220   
21         Forest  0.626685  0.622715         0.817200        0.812482   
22         Forest  0.658487  0.640605         0.831396        0.816558   
23         Forest  0.698811  0.677380         0.842434        0.815691   
24         Forest  0.737752  0.700772         0.866956        0.831315   
25         Forest  0.775377  0.730061         0.895658        0.849343   
26         Forest  0.806197  0.743668         0.923086        0.850713   
27         Forest  0.839455  0.756342         0.952328        0.856584   
28         Forest  0.873328  0.770772         0.972609        0.864717   
29         Forest  0.901664  0.780297         0.986071        0.862061   
30         Forest  0.923960  0.782855         0.993830        0.860561   
31         Forest  0.942671  0.790427         0.997874        0.858123   
32         Forest  0.960485  0.796501         0.999603        0.854285   
33         Forest  0.972359  0.803922         0.999767        0.857007   
34         Forest  0.984531  0.808444         1.000000        0.853692   
35         Forest  0.990853  0.802155         0.999925        0.846930   
36         Forest  0.994576  0.807466         0.999852        0.848497   
37         Forest  0.997018  0.807269         1.000000        0.842546   
38         Forest  0.998235  0.808835         1.000000        0.842239   
39  GradientBoost  0.661789  0.650703         0.811787        0.802222   
40  GradientBoost  0.747041  0.706905         0.844050        0.809344   
41  GradientBoost  0.812011  0.751290         0.891108        0.836743   
42  GradientBoost  0.865537  0.779741         0.928638        0.851184   
43  GradientBoost  0.917434  0.789496         0.962204        0.844315   
44  GradientBoost  0.955398  0.799781         0.984807        0.843929   
45  GradientBoost  0.982758  0.808178         0.996604        0.846529   
46  GradientBoost  0.996723  0.809103         0.999558        0.839411   
47  GradientBoost  0.999302  0.805936         0.999927        0.837870   
48       Logistic  0.630309  0.623306         0.675335        0.671952   
49            SVM  0.632321  0.627337         0.725117        0.718215   
50            SVM  0.718034  0.713356         0.790464        0.787014   
51            SVM  0.790231  0.767116         0.861042        0.835605   
52            SVM  0.867840  0.796932         0.926178        0.846847   
53          

**Random Forest Classifier - Feature Importances for the best parameters**

In [99]:
for (i,key) in enumerate(X.keys()):
    print(key,": ",models[2].steps[1][1].feature_importances_[i])

cv :  0.24277761628274777
spread :  0.17489601977564048
diffp :  0.09803641223294444
rd :  0.08485175058039382
rdnorm :  0.07924220432440161
rdalt :  0.06925518440663439
skew :  0.07604171904893345
ks :  0.12432181339034544
industry_gasoline :  0.013554264140106444
industry_milk :  0.02417712814982537
pricing_pricing :  0.012845887668026922


**Gradient Boosting Classifier - Feature Importances for the parameters model**

In [101]:
for (i,key) in enumerate(X.keys()):
    print(key,": ",models[3].steps[1][1].feature_importances_[i])

cv :  0.3340949930625523
spread :  0.12503567417841477
diffp :  0.11449638351702862
rd :  0.07517277065240258
rdnorm :  0.06867714392604657
rdalt :  0.03820107824554106
skew :  0.07055977249506588
ks :  0.11802197797033225
industry_gasoline :  0.009338307838298351
industry_milk :  0.034329728820489284
pricing_pricing :  0.012072169293828526


# Leave-one-out cross validation

In [40]:
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [41]:
features_ML = ["industry","cv", "spread", "diffp", "rd", "rdnorm", "rdalt", "skew", "ks", "nbids"]
X,y, X_DL = prepare_data("all", features = features_ML, return_industry = True)

  0%|          | 0/5 [00:00<?, ?it/s]

In [42]:
X['industry'].replace(['brazil',"brazil_gaz", 'ticino', "US", "japan" ],
                        [0, 1,2,3,4], inplace=True)

In [43]:
X_DL.shape

(3147, 609, 1)

In [44]:
from sklearn.utils import shuffle

In [45]:
shuffle(np.arange(len(y)))

array([ 708, 1957, 2040, ...,  779, 2190, 2021])

In [46]:
X.keys()

Index(['industry', 'cv', 'spread', 'diffp', 'rd', 'rdnorm', 'rdalt', 'skew',
       'ks', 'nbids'],
      dtype='object')

In [47]:
X_DL[np.asarray([X.industry ==1])[0]].shape

(1438, 609, 1)

In [48]:
X_DL.shape

(3147, 609, 1)

In [49]:
datanames = ['brazil',"brazil_gaz", 'ticino', "US", "japan" ]

In [50]:
groups = X.industry.tolist()

In [63]:
X.drop(labels = ["industry"], axis = "columns")

cv    spread     diffp         rd     rdnorm      rdalt      skew  \
0     0.036899  0.291240  0.000569   0.014614   0.646484   0.645793  1.495001   
1     0.009570  0.049772  0.013699   1.902243   7.155963   9.493671 -2.044359   
2     0.036081  0.285391  0.001699   0.044780   1.970238   1.976048  1.585147   
3     0.034239  0.319767  0.026744   0.727912  27.349091  29.753968  1.326045   
4     0.033923  0.275998  0.000000   0.000000   0.000000   0.000000  1.446969   
...        ...       ...       ...        ...        ...        ...       ...   
3196  0.044565  0.086028  0.075679  10.341437   1.759398   7.312500 -1.637918   
3197  0.047098  0.090968  0.080572  10.960155   1.771429   7.750000 -1.647487   
3198  0.043750  0.091503  0.045752   1.414214   1.000000   1.000000  0.000000   
3199  0.063788  0.134932  0.088356   2.682846   1.309645   1.897059 -0.876676   
3200  0.030396  0.062010  0.039914   2.554708   1.287356   1.806452 -0.820070   

            ks  nbids  
0     5.020408    332  
1     3.543623     27  
2     5.121760    332  
3     5.518785    328  
4     5.572819    328  
...        ...    ...  
3196  1.397671      3  
3197  1.398391      3  
3198  1.250000      3  
3199  1.335997      3  
3200  1.331009      3  

[3147 rows x 9 columns]

In [51]:
logo = LeaveOneGroupOut()


In [52]:
{'GradientBoost__max_depth': 6, 'GradientBoost__random_state': 23} 


{'GradientBoost__max_depth': 6, 'GradientBoost__random_state': 23}

In [53]:
best_model = GradientBoostingClassifier


In [54]:
pipeline = Pipeline([("scaler", StandardScaler()),("GradientBoosting", best_model(max_depth =6, random_state = 23))])


In [55]:
model_scores = []
i=0
logo = LeaveOneGroupOut()
for train, test in logo.split(X, y, groups=groups):
    pipeline = Pipeline([("scaler", StandardScaler()),("GradientBoosting", best_model(max_depth =6, random_state = 23))])
    pipeline.fit(X.iloc[train], y.iloc[train])
    y_pred_train = pipeline.predict(X.iloc[train])
    y_pred_test = pipeline.predict(X.iloc[test])
    
    model_score = pd.DataFrame({"model": "GradientBoosting", "lo_dataset": datanames[i], 
                              "f1_train": ["{:.3f}".format(f1_score(y.iloc[train], y_pred_train))], "f1_test": ["{:.3f}".format(f1_score(y.iloc[test], y_pred_test))],
                              "precision_train":["{:.3f}".format(precision_score(y.iloc[train], y_pred_train))], "precision_test": ["{:.3f}".format(precision_score(y.iloc[test], y_pred_test))],
                              "recall_train": ["{:.3f}".format(recall_score(y.iloc[train], y_pred_train))],"recall_test": ["{:.3f}".format(recall_score(y.iloc[test], y_pred_test))]})
    model_scores.append(model_score)
    i+=1
df_model_scores = pd.concat(model_scores)    
df_model_scores = df_model_scores.reset_index(drop = True)    

In [56]:
df_model_scores

model  lo_dataset f1_train f1_test precision_train  \
0  GradientBoosting      brazil    0.973   0.667           0.992   
1  GradientBoosting  brazil_gaz    0.984   0.074           0.998   
2  GradientBoosting      ticino    0.977   0.492           0.996   
3  GradientBoosting          US    0.998   0.396           0.998   
4  GradientBoosting       japan    0.980   0.176           0.996   

  precision_test recall_train recall_test  
0          0.778        0.954       0.583  
1          0.794        0.971       0.039  
2          0.880        0.958       0.341  
3          0.276        0.998       0.704  
4          0.746        0.965       0.100

In [82]:
df_model_scores

model  lo_dataset  f1_train   f1_test  precision_train  \
0  GradientBoosting      brazil  0.972640  0.666667         0.991632   
1  GradientBoosting  brazil_gaz  0.983990  0.074483         0.997503   
2  GradientBoosting      ticino  0.976813  0.491620         0.996246   
3  GradientBoosting          US  0.998479  0.396040         0.998479   
4  GradientBoosting       japan  0.980029  0.176030         0.996040   

   precision_test  recall_train  recall_test  
0        0.777778      0.954362     0.583333  
1        0.794118      0.970838     0.039074  
2        0.880000      0.958123     0.341085  
3        0.275591      0.998479     0.703518  
4        0.746032      0.964525     0.099788

In [57]:
print(df_model_scores.style.to_latex()) 

\begin{tabular}{lllllllll}
 & model & lo_dataset & f1_train & f1_test & precision_train & precision_test & recall_train & recall_test \\
0 & GradientBoosting & brazil & 0.973 & 0.667 & 0.992 & 0.778 & 0.954 & 0.583 \\
1 & GradientBoosting & brazil_gaz & 0.984 & 0.074 & 0.998 & 0.794 & 0.971 & 0.039 \\
2 & GradientBoosting & ticino & 0.977 & 0.492 & 0.996 & 0.880 & 0.958 & 0.341 \\
3 & GradientBoosting & US & 0.998 & 0.396 & 0.998 & 0.276 & 0.998 & 0.704 \\
4 & GradientBoosting & japan & 0.980 & 0.176 & 0.996 & 0.746 & 0.965 & 0.100 \\
\end{tabular}



In [84]:
full_results

algorithm  train_f1   test_f1  train_precision  test_precision  \
0       Dummy  0.467846  0.471707         0.474867        0.487324   
1        Tree  0.614552  0.612656         0.809077        0.807372   
2        Tree  0.699280  0.697996         0.837032        0.835978   
3        Tree  0.700676  0.694095         0.876362        0.874893   
4        Tree  0.784231  0.777638         0.924868        0.919862   
..        ...       ...       ...              ...             ...   
134       SVM  0.943710  0.792217         0.985866        0.810271   
135       SVM  0.957686  0.784626         0.988673        0.789496   
136       SVM  0.967297  0.777358         0.989121        0.783420   
137       SVM  0.975721  0.773996         0.988701        0.783757   
138       SVM  0.985211  0.748340         0.992630        0.797364   

     train_recall  test_recall  \
0        0.461030     0.457063   
1        0.495450     0.494706   
2        0.600470     0.599730   
3        0.586012     0.579897   
4        0.680758     0.674381   
..            ...          ...   
134      0.905035     0.775410   
135      0.928593     0.780032   
136      0.946426     0.772111   
137      0.963086     0.765511   
138      0.977910     0.706052   

                                                params  
0    {'Dummy__random_state': 23, 'Dummy__strategy':...  
1     {'Tree__max_depth': 1, 'Tree__random_state': 23}  
2     {'Tree__max_depth': 2, 'Tree__random_state': 23}  
3     {'Tree__max_depth': 3, 'Tree__random_state': 23}  
4     {'Tree__max_depth': 4, 'Tree__random_state': 23}  
..                                                 ...  
134  {'SVM__C': 1000.0, 'SVM__gamma': 1.29154966501...  
135  {'SVM__C': 1000.0, 'SVM__gamma': 2.15443469003...  
136  {'SVM__C': 1000.0, 'SVM__gamma': 3.59381366380...  
137  {'SVM__C': 1000.0, 'SVM__gamma': 5.99484250318...  
138  {'SVM__C': 1000.0, 'SVM__gamma': 10.0, 'SVM__k...  

[139 rows x 8 columns]

In [ ]:
classifier.fit(X, y)
        print(f'  - Best hyperparameter: {classifier.best_params_}')
        print(f'  - Best F1 score: {classifier.best_score_:.3f}')
        if print_full:
            print(f'  - F1 Result for each parametrization in train:', [f'{x:.2f}' for x in classifier.cv_results_['mean_train_f1']])
            print(f'  - F1 Result for each parametrization in test:', [f'{x:.2f}' for x in classifier.cv_results_['mean_test_f1']])
            print(f'  - precision Result for each parametrization in train:', [f'{x:.2f}' for x in classifier.cv_results_['mean_train_precision']])
            print(f'  - precision for each parametrization in test:', [f'{x:.2f}' for x in classifier.cv_results_['mean_test_precision']])
            print(f'  - recall Result for each parametrization in train:', [f'{x:.2f}' for x in classifier.cv_results_['mean_train_recall']])
            print(f'  - recall Result for each parametrization in test:', [f'{x:.2f}' for x in classifier.cv_results_['mean_test_recall']])
        model_scores.append(classifier.best_score_)
        models.append(classifier.best_estimator_)
    
        index = classifier.best_index_
        df_results = pd.DataFrame({
            "algorithm": name,
            "train_f1": [classifier.cv_results_['mean_train_f1'][index]], "test_f1": [classifier.cv_results_['mean_test_f1'][index]],
            "train_precision": [classifier.cv_results_['mean_train_precision'][index]], "test_precision": [classifier.cv_results_['mean_test_precision'][index]],
            "train_recall": [classifier.cv_results_['mean_train_recall'][index]], "test_recall": [classifier.cv_results_['mean_test_recall'][index]],
            "params": [classifier.best_params_]
            })
        results.append(df_results)
    
        df_full_results = pd.DataFrame({
            "algorithm": name,
            "train_f1": classifier.cv_results_['mean_train_f1'], "test_f1": classifier.cv_results_['mean_test_f1'],
            "train_precision": classifier.cv_results_['mean_train_precision'], "test_precision": classifier.cv_results_['mean_test_precision'],
            "train_recall": classifier.cv_results_['mean_train_recall'], "test_recall": classifier.cv_results_['mean_test_recall'],
            "params": classifier.cv_results_["params"]
            })
        full_results.append(df_full_results)
    
    results = pd.concat(results)
    results = results.reset_index(drop = True)

    full_results = pd.concat(full_results)
    full_results = full_results.reset_index(drop = True)
    
    return results, full_results

Some sample code found on sklearn

In [ ]:
>>> from sklearn.model_selection import LeaveOneGroupOut

>>> X = [1, 5, 10, 50, 60, 70, 80]
>>> y = [0, 1, 1, 2, 2, 2, 2]
>>> groups = [1, 1, 2, 2, 3, 3, 3]
>>> logo = LeaveOneGroupOut()
>>> for train, test in logo.split(X, y, groups=groups):
...     print("%s %s" % (train, test))
[2 3 4 5 6] [0 1]
[0 1 4 5 6] [2 3]
[0 1 2 3] [4 5 6]

In [55]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


Process dataset such that it can be used in LeaveOneOut Crossvalidation

In [40]:
datanames = [ "Brazil_Gasoline", "japan", "brazil", "ticino", "US_milk"]
features = ["industry", "pricing", "cv", "spread", "diffp", "rd", "rdnorm", "rdalt", "skew", "ks"]
data = []
for name in tqdm(datanames):
    df = pd.read_json(data_dir/f"{name}_train.json")
    if "ID" in df.keys():
        df.rename(columns = {"ID": "project"}, inplace = True)
    if "Price" in df.keys():
        df.rename(columns = {"Price": "bid"}, inplace = True)
    if "complete" in df.keys(): #complete/incomplete is just a case in Swiss dataset, take care of it later
        df.rename(columns = {"complete": "cartel"}, inplace = True)
    df_grouped = df.groupby(by="project").head(1).reset_index()
    df_grouped = df_grouped[["project", "cartel"]+features]
    df_grouped["dataset"] = name
    data.append(df_grouped)
    

df_data = pd.concat(data)
df_data = pd.get_dummies(df_data, columns = ["industry", "pricing"])
df_data.dropna(inplace = True)

  0%|          | 0/5 [00:00<?, ?it/s]

In [41]:
X_data = []
y_data = []
for name in tqdm(datanames):
    X_data.append(df_data[df_data.dataset == name].drop(labels = ["project", "cartel", "industry_construction","pricing_first-price_SBA", "dataset"], axis = "columns"))
    y_data.append(df_data.cartel[df_data.dataset == name])

  0%|          | 0/5 [00:00<?, ?it/s]

In [82]:
model_scores = []
# in models sind die best estimator aus der vorigen Hyperparameter optimisierung erhalten
for (i,model) in enumerate(models):
    print(f'Trying {model.steps[1][0]} ...')
    
    for i in range(len(X_data)):
        
        X_train = pd.concat(np.delete(X_data, i))
        y_train = pd.concat(np.delete(y_data, i))
    
        X_test = X_data[i]
        y_test = y_data[i]
    
        model.fit(X_train, y_train)
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        
        model_score = pd.DataFrame({"model": model.steps[1][0], "lo_dataset": datanames[i], 
                             "f1_test": [f1_score(y_test, y_pred_test)], "f1_train": [f1_score(y_train, y_pred_train)],
                             "precision_test": [precision_score(y_test, y_pred_test)], "precision_train":[precision_score(y_train, y_pred_train)], 
                             "recall_test": [recall_score(y_test, y_pred_test)], "recall_train": [recall_score(y_train, y_pred_train)]})
        model_scores.append(model_score)
df_model_scores = pd.concat(model_scores)    
df_model_scores = df_model_scores.reset_index(drop = True)    

Trying Dummy ...
Trying Tree ...
Trying Forest ...


/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/numpy/lib/function_base.py:5030: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/numpy/lib/function_base.py:5030: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/numpy/lib/function_base.py:5030: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which i

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/numpy/lib/function_base.py:5030: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/numpy/lib/function_base.py:5030: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/numpy/lib/function_base.py:5030: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which i

Trying GradientBoost ...


/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/numpy/lib/function_base.py:5030: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/numpy/lib/function_base.py:5030: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/numpy/lib/function_base.py:5030: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which i

Trying Logistic ...
Trying SVM ...


/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/numpy/lib/function_base.py:5030: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/numpy/lib/function_base.py:5030: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/numpy/lib/function_base.py:5030: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which i

In [84]:
df_model_scores

model       lo_dataset   f1_test  f1_train  precision_test  \
0           Dummy  Brazil_Gasoline  0.455849  0.459900        0.476341   
1           Dummy            japan  0.486005  0.422287        0.606349   
2           Dummy           brazil  0.305085  0.475443        0.257143   
3           Dummy           ticino  0.575758  0.458150        0.826087   
4           Dummy          US_milk  0.389171  0.534134        0.293367   
5            Tree  Brazil_Gasoline  0.337662  0.994509        0.545161   
6            Tree            japan  0.618483  0.969163        0.699732   
7            Tree           brazil  0.615385  0.961155        0.800000   
8            Tree           ticino  0.543478  0.954032        0.909091   
9            Tree          US_milk  0.412831  0.988091        0.285714   
10         Forest  Brazil_Gasoline  0.359212  0.999393        0.901163   
11         Forest            japan  0.478372  0.997115        0.596825   
12         Forest           brazil  0.651163  0.998320        0.736842   
13         Forest           ticino  0.522727  0.993093        0.978723   
14         Forest          US_milk  0.383803  1.000000        0.295393   
15  GradientBoost  Brazil_Gasoline  0.260123  0.999392        0.854839   
16  GradientBoost            japan  0.587007  0.996150        0.647059   
17  GradientBoost           brazil  0.604651  0.995621        0.684211   
18  GradientBoost           ticino  0.625000  0.996740        0.952381   
19  GradientBoost          US_milk  0.402516  1.000000        0.292906   
20       Logistic  Brazil_Gasoline  0.421493  0.701314        0.407008   
21       Logistic            japan  0.735271  0.612521        0.583541   
22       Logistic           brazil  0.080000  0.624687        1.000000   
23       Logistic           ticino  0.395062  0.622951        0.969697   
24       Logistic          US_milk  0.271468  0.678892        0.302469   
25            SVM  Brazil_Gasoline  0.055402  0.822896        0.645161   
26            SVM            japan  0.745791  0.781485        0.617852   
27            SVM           brazil  0.541667  0.829072        0.541667   
28            SVM           ticino  0.898438  0.811283        0.905512   
29            SVM          US_milk  0.440461  0.885400        0.295533   

    precision_train  recall_test  recall_train  
0          0.474774     0.437048      0.445930  
1          0.430708     0.405520      0.414190  
2          0.483356     0.375000      0.467785  
3          0.466019     0.441860      0.450542  
4          0.527017     0.577889      0.541445  
5          0.998775     0.244573      0.990279  
6          0.990000     0.554140      0.949185  
7          0.985201     0.500000      0.938255  
8          0.997636     0.387597      0.914079  
9          0.998447     0.743719      0.977947  
10         0.998786     0.224313      1.000000  
11         1.000000     0.399151      0.994247  
12         0.999328     0.583333      0.997315  
13         1.000000     0.356589      0.986282  
14         1.000000     0.547739      1.000000  
15         1.000000     0.153401      0.998785  
16         1.000000     0.537155      0.992330  
17         0.999324     0.541667      0.991946  
18         1.000000     0.465116      0.993502  
19         1.000000     0.643216      1.000000  
20         0.661638     0.437048      0.746051  
21         0.743836     0.993631      0.520614  
22         0.668966     0.041667      0.585906  
23         0.677995     0.248062      0.576173  
24         0.697115     0.246231      0.661597  
25         0.922961     0.028944      0.742406  
26         0.890798     0.940552      0.696069  
27         0.909455     0.541667      0.761745  
28         0.901943     0.891473      0.737184  
29         0.914842     0.864322      0.857795

## Separate Brasil by city

In [158]:
df_brasil = pd.read_json(data_dir/"brazil_gaz_train.json")
df_brasil_grouped = df_brasil.groupby(by="ID").head(1).reset_index()
df_brasil_grouped.dropna(subset = features)

X = df_brasil_grouped[features]
y = df_brasil_grouped["cartel"]

In [159]:
X = []
y = []
for city in set(df_brasil_grouped.CITY):
    X.append(df_brasil_grouped[df_brasil_grouped.CITY == city][features])
    y.append(df_brasil_grouped.cartel[df_brasil_grouped.CITY == city])

In [166]:
configuration = [
    ('Dummy', DummyClassifier(), {'Dummy__strategy' : ["stratified"], 'Dummy__random_state': [SEED]}),
    ('Tree', DecisionTreeClassifier(), {'Tree__max_depth': range(1,10), 'Tree__random_state': [SEED]}),
    ('Forest', RandomForestClassifier(), {'Forest__max_depth': range(1,20), 'Forest__random_state': [SEED]}),
    ('GradientBoost', GradientBoostingClassifier(), {'GradientBoost__max_depth': range(1,10), 'GradientBoost__random_state': [SEED]}),
    #('MLP', MLPClassifier(), {'MLP__hidden_layer_sizes': range(1,4), "MLP__random_state": [SEED]}),
    ('Logistic', LogisticRegression(), {'Logistic__random_state': [SEED]}),
    #('Poly_SVM', SVC(), {'Poly_SVM__C' : [1.0, 2.5, 5.0, 7.5, 10.0], 'Poly_SVM__kernel' : ['poly'], 'Poly_SVM__degree' : range(1,3), 'Poly_SVM__random_state': [SEED]}),
    ('SVM', SVC(), {'SVM__C' : np.logspace(1,4, 5), 'SVM__gamma': np.logspace(-3,3, 7), 'SVM__kernel' : ['rbf'], 'SVM__random_state': [SEED]})
]

In [181]:
results = []
for i in range(len(X)):
    print("Dataset ", list(set(df_brasil_grouped.CITY))[i])
    model_scores = []
    models = []
    for name, algo, params in configuration:
        print(f'Trying {name} ...')
        # Extend model with normalisation step (here we use z-normalisation)
        pipeline = Pipeline([("scaler", StandardScaler()), (name, algo)])
        # Instantiate grid search object and pass params, number of folds and scoring-function
        classifier = GridSearchCV(pipeline, param_grid=params, cv=StratifiedKFold(n_splits = 5, random_state=SEED, shuffle = True), scoring=['f1', 'precision', 'recall'], refit = 'f1', return_train_score = True)
        # Fit cross validation to our training data: within this function, all configurations are tried
        classifier.fit(X[i], y[i])
        print(f'  - Best hyperparameter: {classifier.best_params_}')
        print(f'  - Best F1 score: {classifier.best_score_:.3f}')
        print(f'  - F1 Result for each parametrization in train:', [f'{x:.2f}' for x in classifier.cv_results_['mean_train_f1']])
        print(f'  - F1 Result for each parametrization in test:', [f'{x:.2f}' for x in classifier.cv_results_['mean_test_f1']])
        print(f'  - precision Result for each parametrization in train:', [f'{x:.2f}' for x in classifier.cv_results_['mean_train_precision']])
        print(f'  - precision for each parametrization in test:', [f'{x:.2f}' for x in classifier.cv_results_['mean_test_precision']])
        print(f'  - recall Result for each parametrization in train:', [f'{x:.2f}' for x in classifier.cv_results_['mean_train_recall']])
        print(f'  - recall Result for each parametrization in test:', [f'{x:.2f}' for x in classifier.cv_results_['mean_test_recall']])
        model_scores.append(classifier.best_score_)
        models.append(classifier.best_estimator_)
        index = classifier.best_index_
        df_results = pd.DataFrame({
            "city": list(set(df_brasil_grouped.CITY))[i], "algorithm": name,
            "train_f1": [classifier.cv_results_['mean_train_f1'][index]], "test_f1": [classifier.cv_results_['mean_test_f1'][index]],
            "train_precision": [classifier.cv_results_['mean_train_precision'][index]], "test_precision": [classifier.cv_results_['mean_test_precision'][index]],
            "train_recall": [classifier.cv_results_['mean_train_recall'][index]], "test_recall": [classifier.cv_results_['mean_test_recall'][index]],
        })
        results.append(df_results)
results = pd.concat(results)
results = results.reset_index(drop = True)    
        

Dataset  BeloHorizonte
Trying Dummy ...
  - Best hyperparameter: {'Dummy__random_state': 23, 'Dummy__strategy': 'stratified'}
  - Best F1 score: 0.464
  - F1 Result for each parametrization in train: ['0.42']
  - F1 Result for each parametrization in test: ['0.46']
  - precision Result for each parametrization in train: ['0.43']
  - precision for each parametrization in test: ['0.47']
  - recall Result for each parametrization in train: ['0.40']
  - recall Result for each parametrization in test: ['0.46']
Trying Tree ...
  - Best hyperparameter: {'Tree__max_depth': 6, 'Tree__random_state': 23}
  - Best F1 score: 0.775
  - F1 Result for each parametrization in train: ['0.39', '0.52', '0.76', '0.77', '0.81', '0.88', '0.91', '0.94', '0.96']
  - F1 Result for each parametrization in test: ['0.38', '0.48', '0.68', '0.68', '0.72', '0.78', '0.77', '0.76', '0.75']
  - precision Result for each parametrization in train: ['0.98', '0.84', '0.64', '0.81', '0.82', '0.83', '0.89', '0.91', '0.95']
  

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

  - Best hyperparameter: {'SVM__C': 10000.0, 'SVM__gamma': 0.01, 'SVM__kernel': 'rbf', 'SVM__random_state': 23}
  - Best F1 score: 0.894
  - F1 Result for each parametrization in train: ['0.64', '0.75', '0.92', '0.98', '1.00', '1.00', '1.00', '0.72', '0.87', '0.94', '1.00', '1.00', '1.00', '1.00', '0.80', '0.92', '0.96', '1.00', '1.00', '1.00', '1.00', '0.86', '0.93', '0.98', '1.00', '1.00', '1.00', '1.00', '0.90', '0.95', '1.00', '1.00', '1.00', '1.00', '1.00']
  - F1 Result for each parametrization in test: ['0.65', '0.74', '0.88', '0.84', '0.58', '0.01', '0.00', '0.72', '0.82', '0.88', '0.84', '0.58', '0.01', '0.00', '0.79', '0.89', '0.89', '0.82', '0.58', '0.01', '0.00', '0.84', '0.89', '0.87', '0.82', '0.58', '0.01', '0.00', '0.87', '0.89', '0.89', '0.82', '0.58', '0.01', '0.00']
  - precision Result for each parametrization in train: ['0.91', '0.84', '0.90', '0.98', '1.00', '1.00', '1.00', '0.86', '0.88', '0.93', '1.00', '1.00', '1.00', '1.00', '0.88', '0.89', '0.96', '1.00', '1.

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

  - Best hyperparameter: {'SVM__C': 56.23413251903491, 'SVM__gamma': 0.1, 'SVM__kernel': 'rbf', 'SVM__random_state': 23}
  - Best F1 score: 0.810
  - F1 Result for each parametrization in train: ['0.61', '0.79', '0.88', '0.98', '1.00', '1.00', '1.00', '0.67', '0.83', '0.93', '1.00', '1.00', '1.00', '1.00', '0.75', '0.84', '0.95', '1.00', '1.00', '1.00', '1.00', '0.80', '0.87', '0.97', '1.00', '1.00', '1.00', '1.00', '0.83', '0.90', '0.99', '1.00', '1.00', '1.00', '1.00']
  - F1 Result for each parametrization in test: ['0.61', '0.77', '0.79', '0.80', '0.64', '0.19', '0.09', '0.64', '0.79', '0.81', '0.79', '0.64', '0.19', '0.09', '0.73', '0.79', '0.78', '0.79', '0.64', '0.19', '0.09', '0.77', '0.79', '0.77', '0.79', '0.64', '0.19', '0.09', '0.77', '0.79', '0.76', '0.79', '0.64', '0.19', '0.09']
  - precision Result for each parametrization in train: ['0.82', '0.84', '0.89', '0.99', '1.00', '1.00', '1.00', '0.82', '0.86', '0.94', '1.00', '1.00', '1.00', '1.00', '0.85', '0.86', '0.97', '1

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

  - Best hyperparameter: {'SVM__C': 10000.0, 'SVM__gamma': 0.01, 'SVM__kernel': 'rbf', 'SVM__random_state': 23}
  - Best F1 score: 0.892
  - F1 Result for each parametrization in train: ['0.68', '0.81', '0.92', '0.98', '1.00', '1.00', '1.00', '0.81', '0.85', '0.93', '0.99', '1.00', '1.00', '1.00', '0.83', '0.89', '0.96', '1.00', '1.00', '1.00', '1.00', '0.87', '0.94', '0.97', '1.00', '1.00', '1.00', '1.00', '0.89', '0.95', '0.98', '1.00', '1.00', '1.00', '1.00']
  - F1 Result for each parametrization in test: ['0.66', '0.81', '0.86', '0.79', '0.47', '0.09', '0.03', '0.77', '0.84', '0.86', '0.80', '0.47', '0.09', '0.03', '0.80', '0.89', '0.85', '0.78', '0.47', '0.09', '0.03', '0.85', '0.87', '0.83', '0.78', '0.47', '0.09', '0.03', '0.89', '0.89', '0.82', '0.78', '0.47', '0.09', '0.03']
  - precision Result for each parametrization in train: ['0.88', '0.84', '0.95', '0.98', '1.00', '1.00', '1.00', '0.84', '0.90', '0.95', '0.99', '1.00', '1.00', '1.00', '0.86', '0.94', '0.96', '1.00', '1.

In [193]:
results

city      algorithm  train_f1   test_f1  train_precision  \
0   BeloHorizonte          Dummy  0.418286  0.463772         0.433394   
1   BeloHorizonte           Tree  0.875835  0.775041         0.834561   
2   BeloHorizonte         Forest  1.000000  0.852692         1.000000   
3   BeloHorizonte  GradientBoost  1.000000  0.871221         1.000000   
4   BeloHorizonte       Logistic  0.783056  0.783591         0.838113   
5   BeloHorizonte            SVM  0.945346  0.893650         0.934595   
6         SaoLuis          Dummy  0.455928  0.432727         0.467016   
7         SaoLuis           Tree  0.979841  0.782293         0.993548   
8         SaoLuis         Forest  0.967481  0.819664         0.974857   
9         SaoLuis  GradientBoost  0.959829  0.834569         0.969761   
10        SaoLuis       Logistic  0.669484  0.621607         0.784271   
11        SaoLuis            SVM  0.929455  0.809659         0.944137   
12    CaxiasdoSul          Dummy  0.334729  0.361469         0.363636   
13    CaxiasdoSul           Tree  0.919901  0.751141         0.937931   
14    CaxiasdoSul         Forest  0.999024  0.829433         1.000000   
15    CaxiasdoSul  GradientBoost  0.999024  0.827810         1.000000   
16    CaxiasdoSul       Logistic  0.817536  0.795596         0.867542   
17    CaxiasdoSul            SVM  0.947108  0.891596         0.958736   
18       Brasilia          Dummy  0.681460  0.691185         0.656846   
19       Brasilia           Tree  0.993870  0.951224         0.990868   
20       Brasilia         Forest  0.994911  0.971588         0.989883   
21       Brasilia  GradientBoost  1.000000  0.969486         1.000000   
22       Brasilia       Logistic  0.938099  0.927958         0.920967   
23       Brasilia            SVM  0.985229  0.971592         0.979757   

    test_precision  train_recall  test_recall  
0         0.466667      0.404197     0.460963  
1         0.738349      0.926731     0.820677  
2         0.872862      1.000000     0.839216  
3         0.890899      1.000000     0.856684  
4         0.836988      0.735024     0.743316  
5         0.883599      0.956593     0.904635  
6         0.428602      0.445358     0.436989  
7         0.788371      0.966915     0.781720  
8         0.838835      0.960262     0.807312  
9         0.839028      0.950331     0.834624  
10        0.749907      0.584463     0.537204  
11        0.818769      0.915565     0.808387  
12        0.375000      0.310082     0.348923  
13        0.775191      0.903099     0.743077  
14        0.860794      0.998058     0.813231  
15        0.850367      0.998058     0.820615  
16        0.856500      0.773264     0.750769  
17        0.921812      0.936034     0.867692  
18        0.689796      0.707991     0.692602  
19        0.943893      0.996923     0.959099  
20        0.964367      1.000000     0.979592  
21        0.964225      1.000000     0.975510  
22        0.907544      0.955924     0.951020  
23        0.963994      0.990785     0.979592

# Overfit Tree

Give information about how contradictory data is, if it is possible to seperate it perfectly. Still this is a bit fishy, as with only real numbers features, it can always be arbitrary seperated.

In [20]:
configuration = [
    ('Tree', DecisionTreeClassifier(), {'Tree__max_depth': [10,20,30,40,50,100,150,200,250,300,350,500,1000], 'Tree__random_state': [SEED]})
]

In [23]:
model_scores = []
models = []
results = []
full_results = []
for name, algo, params in configuration:
    print(f'Trying {name} ...')
    # Extend model with normalisation step (here we use z-normalisation)
    pipeline = Pipeline([("scaler", StandardScaler()), (name, algo)])
    # Instantiate grid search object and pass params, number of folds and scoring-function
    classifier = GridSearchCV(pipeline, param_grid=params, cv=StratifiedKFold(n_splits = 10, random_state=SEED, shuffle = True), scoring=['f1', 'precision', 'recall'], refit = 'f1', return_train_score = True)
    # Fit cross validation to our training data: within this function, all configurations are tried
    classifier.fit(X, y)
    print(f'  - Best hyperparameter: {classifier.best_params_}')
    print(f'  - Best F1 score: {classifier.best_score_:.3f}')
    print(f'  - F1 Result for each parametrization in train:', [f'{x:.2f}' for x in classifier.cv_results_['mean_train_f1']])
    print(f'  - F1 Result for each parametrization in test:', [f'{x:.2f}' for x in classifier.cv_results_['mean_test_f1']])
    print(f'  - precision Result for each parametrization in train:', [f'{x:.2f}' for x in classifier.cv_results_['mean_train_precision']])
    print(f'  - precision for each parametrization in test:', [f'{x:.2f}' for x in classifier.cv_results_['mean_test_precision']])
    print(f'  - recall Result for each parametrization in train:', [f'{x:.2f}' for x in classifier.cv_results_['mean_train_recall']])
    print(f'  - recall Result for each parametrization in test:', [f'{x:.2f}' for x in classifier.cv_results_['mean_test_recall']])
    model_scores.append(classifier.best_score_)
    models.append(classifier.best_estimator_)
    
    index = classifier.best_index_
    df_results = pd.DataFrame({
            "algorithm": name,
            "train_f1": [classifier.cv_results_['mean_train_f1'][index]], "test_f1": [classifier.cv_results_['mean_test_f1'][index]],
            "train_precision": [classifier.cv_results_['mean_train_precision'][index]], "test_precision": [classifier.cv_results_['mean_test_precision'][index]],
            "train_recall": [classifier.cv_results_['mean_train_recall'][index]], "test_recall": [classifier.cv_results_['mean_test_recall'][index]],
            "params": [classifier.best_params_]
        })
    results.append(df_results)
    
    df_full_results = pd.DataFrame({
            "algorithm": name,
            "train_f1": classifier.cv_results_['mean_train_f1'], "test_f1": classifier.cv_results_['mean_test_f1'],
            "train_precision": classifier.cv_results_['mean_train_precision'], "test_precision": classifier.cv_results_['mean_test_precision'],
            "train_recall": classifier.cv_results_['mean_train_recall'], "test_recall": classifier.cv_results_['mean_test_recall'],
            "params": classifier.cv_results_["params"]
        })
    full_results.append(df_full_results)
    
results = pd.concat(results)
results = results.reset_index(drop = True)

full_results = pd.concat(full_results)
full_results = full_results.reset_index(drop = True)

Trying Tree ...
  - Best hyperparameter: {'Tree__max_depth': 30, 'Tree__random_state': 23}
  - Best F1 score: 0.753
  - F1 Result for each parametrization in train: ['0.83', '0.98', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00']
  - F1 Result for each parametrization in test: ['0.73', '0.74', '0.75', '0.75', '0.75', '0.75', '0.75', '0.75', '0.75', '0.75', '0.75', '0.75', '0.75']
  - precision Result for each parametrization in train: ['0.91', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00']
  - precision for each parametrization in test: ['0.80', '0.76', '0.75', '0.75', '0.75', '0.75', '0.75', '0.75', '0.75', '0.75', '0.75', '0.75', '0.75']
  - recall Result for each parametrization in train: ['0.77', '0.97', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00']
  - recall Result for each parametrization in test: ['0.67', '0.73', '0.76', '0.75', '0.75', '0.75', '0.7

**Japan**

F1 Result for each parametrization in train: ['0.99', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00'

**Brazil**

F1 Result for each parametrization in train: ['0.93', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00']

**All data together**

F1 Result for each parametrization in train: ['0.83', '0.98', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00']

**Summary**

Basically does not say anything, as we only have floating point number features. 

# Automatic Feature Engineering

In [16]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier

## Feature Generation

Load Data

In [59]:
features_ML = ["cv", "spread", "diffp", "rd", "rdnorm", "rdalt", "skew", "ks", "nbids"]

In [60]:
X,y,_ = prepare_data("brazil_gaz", features = features_ML)

**Generate Polynomial Features**

In [61]:
poly = PolynomialFeatures(2) #second order polynomial use all interactions

X_mf = pd.DataFrame()
X_mf[poly.get_feature_names_out(features_ML)] = poly.fit_transform(X)
X_mf.head()

1        cv    spread     diffp        rd     rdnorm      rdalt  \
0  1.0  0.036899  0.291240  0.000569  0.014614   0.646484   0.645793   
1  1.0  0.009570  0.049772  0.013699  1.902243   7.155963   9.493671   
2  1.0  0.036081  0.285391  0.001699  0.044780   1.970238   1.976048   
3  1.0  0.034239  0.319767  0.026744  0.727912  27.349091  29.753968   
4  1.0  0.033923  0.275998  0.000000  0.000000   0.000000   0.000000   

       skew        ks  nbids  ...     rdalt^2  rdalt skew    rdalt ks  \
0  1.495001  5.020408  332.0  ...    0.417048    0.965461    3.242142   
1 -2.044359  3.543623   27.0  ...   90.129787  -19.408471   33.641989   
2  1.585147  5.121760  332.0  ...    3.904765    3.132326   10.120844   
3  1.326045  5.518785  328.0  ...  885.298627   39.455100  164.205756   
4  1.446969  5.572819  328.0  ...    0.000000    0.000000    0.000000   

   rdalt nbids    skew^2   skew ks  skew nbids       ks^2     ks nbids  \
0   214.403131  2.235028  7.505515  496.340346  25.204496  1666.775450   
1   256.329114  4.179404 -7.244437  -55.197692  12.557263    95.677817   
2   656.047904  2.512690  8.118741  526.268686  26.232429  1700.424435   
3  9759.301587  1.758395  7.318157  434.942750  30.456989  1810.161502   
4     0.000000  2.093719  8.063695  474.605763  31.056308  1827.884512   

    nbids^2  
0  110224.0  
1     729.0  
2  110224.0  
3  107584.0  
4  107584.0  

[5 rows x 55 columns]

**Add non-polynnomial features**

In [62]:
transforms = [np.sqrt, np.log1p, np.sin, np.cos]
transforms_names = ["sqrt", "log1p", "sin", "cos"]

In [63]:
for stransform,name in zip(transforms,transforms_names):
    transformer = FunctionTransformer(stransform, validate=True, feature_names_out='one-to-one')
    
    if name in ["sqrt", "log1p"]:
        X_mf[transformer.get_feature_names_out([f"{name}_{fname}" for fname in features_ML])] = transformer.transform(abs(X))
    else:
        X_mf[transformer.get_feature_names_out([f"{name}_{fname}" for fname in features_ML])] = transformer.transform(X) 
    

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but FunctionTransformer was fitted without feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but FunctionTransformer was fitted without feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but FunctionTransformer was fitted without feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but FunctionTransformer was fitted without feature names
  warnings.warn(


In [64]:
X_mf.head()

1        cv    spread     diffp        rd     rdnorm      rdalt  \
0  1.0  0.036899  0.291240  0.000569  0.014614   0.646484   0.645793   
1  1.0  0.009570  0.049772  0.013699  1.902243   7.155963   9.493671   
2  1.0  0.036081  0.285391  0.001699  0.044780   1.970238   1.976048   
3  1.0  0.034239  0.319767  0.026744  0.727912  27.349091  29.753968   
4  1.0  0.033923  0.275998  0.000000  0.000000   0.000000   0.000000   

       skew        ks  nbids  ...  sin_nbids    cos_cv  cos_spread  cos_diffp  \
0  1.495001  5.020408  332.0  ...  -0.846204  0.999319    0.957889   1.000000   
1 -2.044359  3.543623   27.0  ...   0.956376  0.999954    0.998762   0.999906   
2  1.585147  5.121760  332.0  ...  -0.846204  0.999349    0.959552   0.999999   
3  1.326045  5.518785  328.0  ...   0.956385  0.999414    0.949309   0.999642   
4  1.446969  5.572819  328.0  ...   0.956385  0.999425    0.962154   1.000000   

     cos_rd  cos_rdnorm  cos_rdalt  cos_skew    cos_ks  cos_nbids  
0  0.999893    0.798206   0.798623  0.075723  0.303171   0.532859  
1 -0.325411    0.642701  -0.997628 -0.456060 -0.920268  -0.292139  
2  0.998998   -0.388904  -0.394250 -0.014350  0.398033   0.532859  
3  0.746565   -0.601641  -0.091036  0.242315  0.721798   0.292110  
4  1.000000    1.000000   1.000000  0.123511  0.758123   0.292110  

[5 rows x 91 columns]

In [65]:
features_in = X_mf.keys()

In [66]:
print(features_in)

Index(['1', 'cv', 'spread', 'diffp', 'rd', 'rdnorm', 'rdalt', 'skew', 'ks',
       'nbids', 'cv^2', 'cv spread', 'cv diffp', 'cv rd', 'cv rdnorm',
       'cv rdalt', 'cv skew', 'cv ks', 'cv nbids', 'spread^2', 'spread diffp',
       'spread rd', 'spread rdnorm', 'spread rdalt', 'spread skew',
       'spread ks', 'spread nbids', 'diffp^2', 'diffp rd', 'diffp rdnorm',
       'diffp rdalt', 'diffp skew', 'diffp ks', 'diffp nbids', 'rd^2',
       'rd rdnorm', 'rd rdalt', 'rd skew', 'rd ks', 'rd nbids', 'rdnorm^2',
       'rdnorm rdalt', 'rdnorm skew', 'rdnorm ks', 'rdnorm nbids', 'rdalt^2',
       'rdalt skew', 'rdalt ks', 'rdalt nbids', 'skew^2', 'skew ks',
       'skew nbids', 'ks^2', 'ks nbids', 'nbids^2', 'sqrt_cv', 'sqrt_spread',
       'sqrt_diffp', 'sqrt_rd', 'sqrt_rdnorm', 'sqrt_rdalt', 'sqrt_skew',
       'sqrt_ks', 'sqrt_nbids', 'log1p_cv', 'log1p_spread', 'log1p_diffp',
       'log1p_rd', 'log1p_rdnorm', 'log1p_rdalt', 'log1p_skew', 'log1p_ks',
       'log1p_nbids', 'sin_cv', 's

In [67]:
len(features_in)

91

## Feature Selection

Probably I have some information leak if I use all the data to do feature selection?

In [68]:
from sklearn.svm import LinearSVC

In [69]:
scaler = StandardScaler()
scaler.fit(X_mf)
X_mf = scaler.transform(X_mf)

In [70]:
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False, random_state = SEED).fit(X_mf, y)

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [71]:
model = SelectFromModel(lsvc, prefit = True) 

**TODO: Read about what this is exactly doing**

In [72]:
X_new = model.transform(X_mf)
X_new.shape

(1438, 15)

In [73]:
features_out = model.get_feature_names_out(features_in)

In [74]:
print(features_out)

['nbids' 'cv^2' 'ks nbids' 'sqrt_cv' 'sqrt_diffp' 'sqrt_ks' 'sqrt_nbids'
 'log1p_ks' 'log1p_nbids' 'sin_skew' 'sin_nbids' 'cos_cv' 'cos_rd'
 'cos_skew' 'cos_nbids']


**Interesting what features seem to be important !!!**

Let's see now whether the performance increases

In [75]:
print(X_new.shape)
print(y.shape)

(1438, 15)
(1438,)


In [76]:
results, full_results = evaluate_models(X_new, y, conf_ML, SEED = SEED)

Trying Dummy ...
  - Best hyperparameter: {'Dummy__random_state': 23, 'Dummy__strategy': 'stratified'}
  - Best F1 score: 0.470
Trying Tree ...
  - Best hyperparameter: {'Tree__max_depth': 5, 'Tree__random_state': 23}
  - Best F1 score: 0.920
Trying Forest ...
  - Best hyperparameter: {'Forest__max_depth': 17, 'Forest__random_state': 23}
  - Best F1 score: 0.960
Trying GradientBoost ...
  - Best hyperparameter: {'GradientBoost__max_depth': 5, 'GradientBoost__random_state': 23}
  - Best F1 score: 0.961
Trying MLP ...


/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum itera

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum itera

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum itera

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum itera

...

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

  - Best hyperparameter: {'Poly_SVM__C': 545.5594781168514, 'Poly_SVM__degree': 3, 'Poly_SVM__kernel': 'poly', 'Poly_SVM__random_state': 23}
  - Best F1 score: 0.906
Trying SVM ...
  - Best hyperparameter: {'SVM__C': 78.47599703514611, 'SVM__gamma': 0.1, 'SVM__kernel': 'rbf', 'SVM__random_state': 23}
  - Best F1 score: 0.929


**All Data**

algorithm train_f1 test_f1 train_precision test_precision train_recall  \
0          Dummy    0.450   0.470           0.470          0.482        0.431   
1           Tree    0.972   0.933           0.966          0.932        0.978   
2         Forest    1.000   0.936           1.000          0.957        1.000   
3  GradientBoost    1.000   0.963           1.000          0.970        1.000   
4            MLP    0.852   0.843           0.898          0.887        0.810   
5       Logistic    0.805   0.797           0.834          0.823        0.778   
6       Poly_SVM    0.861   0.846           0.910          0.897        0.817   
7            SVM    0.972   0.881           0.982          0.884        0.961   

  test_recall train_accuracy test_accuracy  \
0       0.459          0.493         0.503   
1       0.935          0.973         0.935   
2       0.916          1.000         0.939   
3       0.957          1.000         0.965   
4       0.805          0.865         0.856   
5       0.774          0.819         0.811   
6       0.802          0.873         0.860   
7       0.878          0.973         0.886   

                                              params  
0  {'Dummy__random_state': 23, 'Dummy__strategy':...  
1   {'Tree__max_depth': 8, 'Tree__random_state': 23}  
2  {'Forest__max_depth': 16, 'Forest__random_stat...  
3  {'GradientBoost__max_depth': 5, 'GradientBoost...  
4  {'MLP__hidden_layer_sizes': 17, 'MLP__random_s...  
5  {'Logistic__C': 21.544346900318832, 'Logistic_...  
6  {'Poly_SVM__C': 3792.690190732246, 'Poly_SVM__...  
7  {'SVM__C': 29.76351441631318, 'SVM__gamma': 0....

In [189]:
results

algorithm  train_f1   test_f1  train_precision  test_precision  \
0          Dummy  0.467846  0.471707         0.474867        0.487324   
1           Tree  0.910802  0.827473         0.971932        0.890071   
2         Forest  0.945954  0.863160         0.996106        0.919036   
3  GradientBoost  0.999706  0.861691         0.999853        0.887930   
4       Logistic  0.741568  0.737140         0.796012        0.790652   
5            SVM  0.930336  0.812544         0.982299        0.861295   

   train_recall  test_recall  \
0      0.461030     0.457063   
1      0.857335     0.774137   
2      0.900632     0.814417   
3      0.999560     0.837565   
4      0.694115     0.690894   
5      0.883606     0.769545   

                                              params  
0  {'Dummy__random_state': 23, 'Dummy__strategy':...  
1   {'Tree__max_depth': 9, 'Tree__random_state': 23}  
2  {'Forest__max_depth': 11, 'Forest__random_stat...  
3  {'GradientBoost__max_depth': 9, 'GradientBoost...  
4                     {'Logistic__random_state': 23}  
5  {'SVM__C': 129.1549665014884, 'SVM__gamma': 0....

In [190]:
print(results.to_latex(index=False))

\begin{tabular}{lrrrrrrl}
\toprule
    algorithm &  train\_f1 &  test\_f1 &  train\_precision &  test\_precision &  train\_recall &  test\_recall &                                             params \\
\midrule
        Dummy &  0.467846 & 0.471707 &         0.474867 &        0.487324 &      0.461030 &     0.457063 & \{'Dummy\_\_random\_state': 23, 'Dummy\_\_strategy': ... \\
         Tree &  0.910802 & 0.827473 &         0.971932 &        0.890071 &      0.857335 &     0.774137 &   \{'Tree\_\_max\_depth': 9, 'Tree\_\_random\_state': 23\} \\
       Forest &  0.945954 & 0.863160 &         0.996106 &        0.919036 &      0.900632 &     0.814417 & \{'Forest\_\_max\_depth': 11, 'Forest\_\_random\_state... \\
GradientBoost &  0.999706 & 0.861691 &         0.999853 &        0.887930 &      0.999560 &     0.837565 & \{'GradientBoost\_\_max\_depth': 9, 'GradientBoost\_... \\
     Logistic &  0.741568 & 0.737140 &         0.796012 &        0.790652 &      0.694115 &     0.690894 &             

/var/folders/kv/34jyv69d3lq7qnz8d_2l_s4w0000gq/T/ipykernel_22684/2695231829.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results.to_latex(index=False))


**Brazil Data**

**Results of reevaluation on 16. January**

In [77]:
results

algorithm train_f1 test_f1 train_precision test_precision train_recall  \
0          Dummy    0.450   0.470           0.470          0.482        0.431   
1           Tree    0.941   0.920           0.938          0.921        0.944   
2         Forest    1.000   0.960           1.000          0.964        1.000   
3  GradientBoost    1.000   0.961           1.000          0.964        1.000   
4            MLP    0.896   0.877           0.922          0.900        0.871   
5       Logistic    0.836   0.835           0.871          0.874        0.804   
6       Poly_SVM    0.981   0.906           0.984          0.923        0.978   
7            SVM    0.991   0.929           0.992          0.930        0.991   

  test_recall train_accuracy test_accuracy  \
0       0.459          0.493         0.503   
1       0.920          0.943         0.923   
2       0.957          1.000         0.962   
3       0.958          1.000         0.962   
4       0.855          0.903         0.885   
5       0.802          0.849         0.848   
6       0.891          0.982         0.912   
7       0.929          0.992         0.932   

                                              params  
0  {'Dummy__random_state': 23, 'Dummy__strategy':...  
1   {'Tree__max_depth': 5, 'Tree__random_state': 23}  
2  {'Forest__max_depth': 17, 'Forest__random_stat...  
3  {'GradientBoost__max_depth': 5, 'GradientBoost...  
4  {'MLP__hidden_layer_sizes': 18, 'MLP__random_s...  
5  {'Logistic__C': 3593.813663804626, 'Logistic__...  
6  {'Poly_SVM__C': 545.5594781168514, 'Poly_SVM__...  
7  {'SVM__C': 78.47599703514611, 'SVM__gamma': 0....

In [78]:
print(results.to_latex(index=False))

\begin{tabular}{llllllllll}
\toprule
    algorithm & train\_f1 & test\_f1 & train\_precision & test\_precision & train\_recall & test\_recall & train\_accuracy & test\_accuracy &                                             params \\
\midrule
        Dummy &    0.450 &   0.470 &           0.470 &          0.482 &        0.431 &       0.459 &          0.493 &         0.503 & \{'Dummy\_\_random\_state': 23, 'Dummy\_\_strategy': ... \\
         Tree &    0.941 &   0.920 &           0.938 &          0.921 &        0.944 &       0.920 &          0.943 &         0.923 &   \{'Tree\_\_max\_depth': 5, 'Tree\_\_random\_state': 23\} \\
       Forest &    1.000 &   0.960 &           1.000 &          0.964 &        1.000 &       0.957 &          1.000 &         0.962 & \{'Forest\_\_max\_depth': 17, 'Forest\_\_random\_state... \\
GradientBoost &    1.000 &   0.961 &           1.000 &          0.964 &        1.000 &       0.958 &          1.000 &         0.962 & \{'GradientBoost\_\_max\_depth': 5, 'Gr

/var/folders/kv/34jyv69d3lq7qnz8d_2l_s4w0000gq/T/ipykernel_1359/2695231829.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results.to_latex(index=False))


**Results of original Evaluation**

In [57]:
results

algorithm  train_f1   test_f1  train_precision  test_precision  \
0          Dummy  0.449869  0.470000         0.470351        0.481795   
1           Tree  0.941343  0.919944         0.938420        0.921037   
2         Forest  1.000000  0.960080         1.000000        0.964491   
3  GradientBoost  1.000000  0.960848         1.000000        0.964294   
4       Logistic  0.827168  0.823203         0.868993        0.865105   
5            SVM  0.993334  0.925419         0.992302        0.928210   

   train_recall  test_recall  \
0      0.431097     0.458778   
1      0.944365     0.920414   
2      1.000000     0.956625   
3      1.000000     0.958054   
4      0.789194     0.787226   
5      0.994372     0.923313   

                                              params  
0  {'Dummy__random_state': 23, 'Dummy__strategy':...  
1   {'Tree__max_depth': 5, 'Tree__random_state': 23}  
2  {'Forest__max_depth': 17, 'Forest__random_stat...  
3  {'GradientBoost__max_depth': 5, 'GradientBoost...  
4                     {'Logistic__random_state': 23}  
5  {'SVM__C': 100.0, 'SVM__gamma': 0.1, 'SVM__ker...

In [31]:
results

algorithm train_f1 test_f1 train_precision test_precision train_recall  \
0          Dummy    0.450   0.470           0.470          0.482        0.431   
1           Tree    0.941   0.920           0.938          0.921        0.944   
2         Forest    1.000   0.960           1.000          0.964        1.000   
3  GradientBoost    1.000   0.961           1.000          0.964        1.000   
4            MLP    0.896   0.877           0.922          0.900        0.871   
5       Logistic    0.836   0.835           0.871          0.874        0.804   
6            SVM    0.991   0.929           0.992          0.930        0.991   

  test_recall                                             params  
0       0.459  {'Dummy__random_state': 23, 'Dummy__strategy':...  
1       0.920   {'Tree__max_depth': 5, 'Tree__random_state': 23}  
2       0.957  {'Forest__max_depth': 17, 'Forest__random_stat...  
3       0.958  {'GradientBoost__max_depth': 5, 'GradientBoost...  
4       0.855  {'MLP__hidden_layer_sizes': 18, 'MLP__random_s...  
5       0.802  {'Logistic__C': 3593.813663804626, 'Logistic__...  
6       0.929  {'SVM__C': 78.47599703514611, 'SVM__gamma': 0....

In [58]:
print(results.to_latex(index=False))

\begin{tabular}{lrrrrrrl}
\toprule
    algorithm &  train\_f1 &  test\_f1 &  train\_precision &  test\_precision &  train\_recall &  test\_recall &                                             params \\
\midrule
        Dummy &  0.449869 & 0.470000 &         0.470351 &        0.481795 &      0.431097 &     0.458778 & \{'Dummy\_\_random\_state': 23, 'Dummy\_\_strategy': ... \\
         Tree &  0.941343 & 0.919944 &         0.938420 &        0.921037 &      0.944365 &     0.920414 &   \{'Tree\_\_max\_depth': 5, 'Tree\_\_random\_state': 23\} \\
       Forest &  1.000000 & 0.960080 &         1.000000 &        0.964491 &      1.000000 &     0.956625 & \{'Forest\_\_max\_depth': 17, 'Forest\_\_random\_state... \\
GradientBoost &  1.000000 & 0.960848 &         1.000000 &        0.964294 &      1.000000 &     0.958054 & \{'GradientBoost\_\_max\_depth': 5, 'GradientBoost\_... \\
     Logistic &  0.827168 & 0.823203 &         0.868993 &        0.865105 &      0.789194 &     0.787226 &             

/var/folders/kv/34jyv69d3lq7qnz8d_2l_s4w0000gq/T/ipykernel_23340/2695231829.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results.to_latex(index=False))


In [34]:
print(results[["algorithm", "train_f1", "test_f1", "train_precision", "test_precision", "train_recall", "test_recall"]].to_latex(index=False))

\begin{tabular}{lllllll}
\toprule
    algorithm & train\_f1 & test\_f1 & train\_precision & test\_precision & train\_recall & test\_recall \\
\midrule
        Dummy &    0.450 &   0.470 &           0.470 &          0.482 &        0.431 &       0.459 \\
         Tree &    0.941 &   0.920 &           0.938 &          0.921 &        0.944 &       0.920 \\
       Forest &    1.000 &   0.960 &           1.000 &          0.964 &        1.000 &       0.957 \\
GradientBoost &    1.000 &   0.961 &           1.000 &          0.964 &        1.000 &       0.958 \\
          MLP &    0.896 &   0.877 &           0.922 &          0.900 &        0.871 &       0.855 \\
     Logistic &    0.836 &   0.835 &           0.871 &          0.874 &        0.804 &       0.802 \\
          SVM &    0.991 &   0.929 &           0.992 &          0.930 &        0.991 &       0.929 \\
\bottomrule
\end{tabular}



/var/folders/kv/34jyv69d3lq7qnz8d_2l_s4w0000gq/T/ipykernel_86900/446016404.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results[["algorithm", "train_f1", "test_f1", "train_precision", "test_precision", "train_recall", "test_recall"]].to_latex(index=False))


## Selection via Tree Model

In [36]:
clf = ExtraTreesClassifier(n_estimators=100, random_state = SEED)
clf = clf.fit(X_mf, y)
print(clf.feature_importances_) 
model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X_mf)

[0.         0.023504   0.01630082 0.00589235 0.00375576 0.00423217
 0.00342981 0.00679806 0.0135935  0.04289717 0.01648327 0.01547646
 0.00644287 0.00755766 0.00388443 0.00304572 0.00671465 0.01493615
 0.02958319 0.01415664 0.00528564 0.00442386 0.00336311 0.00357181
 0.00665596 0.00703295 0.0250655  0.00455298 0.00414873 0.00279654
 0.00319523 0.00352354 0.00482457 0.00356842 0.00313984 0.00319116
 0.00263859 0.00381387 0.00384621 0.00288356 0.00387597 0.00270214
 0.00391201 0.0047515  0.00922899 0.00262182 0.00320164 0.00332272
 0.00669441 0.00979215 0.00816987 0.01142667 0.01545509 0.03021173
 0.04132975 0.02886097 0.01480741 0.00763611 0.00514039 0.00452532
 0.00387693 0.01295529 0.01127814 0.05251255 0.02695095 0.0179125
 0.00520218 0.00410696 0.00501303 0.00461309 0.00951316 0.01282275
 0.05004879 0.02545814 0.01466563 0.0051704  0.00372819 0.0043771
 0.00422463 0.00839296 0.01211528 0.03578947 0.02224664 0.01112982
 0.00540419 0.00422494 0.00357741 0.00437342 0.01154561 0.009761

In [37]:
results, full_results = evaluate_models(X_new, y, conf_ML, SEED = SEED)

Trying Dummy ...
  - Best hyperparameter: {'Dummy__random_state': 23, 'Dummy__strategy': 'stratified'}
  - Best F1 score: 0.470
Trying Tree ...
  - Best hyperparameter: {'Tree__max_depth': 9, 'Tree__random_state': 23}
  - Best F1 score: 0.929
Trying Forest ...
  - Best hyperparameter: {'Forest__max_depth': 15, 'Forest__random_state': 23}
  - Best F1 score: 0.959
Trying GradientBoost ...
  - Best hyperparameter: {'GradientBoost__max_depth': 5, 'GradientBoost__random_state': 23}
  - Best F1 score: 0.959
Trying MLP ...


/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum itera

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum itera

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum itera

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum itera

...

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/eventfrog/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt

  - Best hyperparameter: {'Poly_SVM__C': 1438.449888287663, 'Poly_SVM__degree': 3, 'Poly_SVM__kernel': 'poly', 'Poly_SVM__random_state': 23}
  - Best F1 score: 0.921
Trying SVM ...
  - Best hyperparameter: {'SVM__C': 29.76351441631318, 'SVM__gamma': 0.1, 'SVM__kernel': 'rbf', 'SVM__random_state': 23}
  - Best F1 score: 0.931


**All**

In [195]:
results

algorithm  train_f1   test_f1  train_precision  test_precision  \
0          Dummy  0.467846  0.471707         0.474867        0.487324   
1           Tree  0.908832  0.842662         0.960315        0.898608   
2         Forest  0.968827  0.866253         0.999452        0.906303   
3  GradientBoost  0.994249  0.864173         0.998594        0.887039   
4       Logistic  0.736360  0.732795         0.780011        0.775851   
5            SVM  0.908144  0.838484         0.968180        0.892296   

   train_recall  test_recall  \
0      0.461030     0.457063   
1      0.862984     0.793935   
2      0.940042     0.830263   
3      0.989946     0.842833   
4      0.697343     0.694872   
5      0.855131     0.791356   

                                              params  
0  {'Dummy__random_state': 23, 'Dummy__strategy':...  
1   {'Tree__max_depth': 9, 'Tree__random_state': 23}  
2  {'Forest__max_depth': 14, 'Forest__random_stat...  
3  {'GradientBoost__max_depth': 7, 'GradientBoost...  
4                     {'Logistic__random_state': 23}  
5  {'SVM__C': 129.1549665014884, 'SVM__gamma': 0....

In [196]:
print(results.to_latex(index=False))

\begin{tabular}{lrrrrrrl}
\toprule
    algorithm &  train\_f1 &  test\_f1 &  train\_precision &  test\_precision &  train\_recall &  test\_recall &                                             params \\
\midrule
        Dummy &  0.467846 & 0.471707 &         0.474867 &        0.487324 &      0.461030 &     0.457063 & \{'Dummy\_\_random\_state': 23, 'Dummy\_\_strategy': ... \\
         Tree &  0.908832 & 0.842662 &         0.960315 &        0.898608 &      0.862984 &     0.793935 &   \{'Tree\_\_max\_depth': 9, 'Tree\_\_random\_state': 23\} \\
       Forest &  0.968827 & 0.866253 &         0.999452 &        0.906303 &      0.940042 &     0.830263 & \{'Forest\_\_max\_depth': 14, 'Forest\_\_random\_state... \\
GradientBoost &  0.994249 & 0.864173 &         0.998594 &        0.887039 &      0.989946 &     0.842833 & \{'GradientBoost\_\_max\_depth': 7, 'GradientBoost\_... \\
     Logistic &  0.736360 & 0.732795 &         0.780011 &        0.775851 &      0.697343 &     0.694872 &             

/var/folders/kv/34jyv69d3lq7qnz8d_2l_s4w0000gq/T/ipykernel_22684/2695231829.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results.to_latex(index=False))


**Brazil Gaz**

**results of reevaluation on 16. Jan 2024**

In [38]:
results

algorithm train_f1 test_f1 train_precision test_precision train_recall  \
0          Dummy    0.450   0.470           0.470          0.482        0.431   
1           Tree    0.986   0.929           0.986          0.931        0.987   
2         Forest    1.000   0.959           1.000          0.963        1.000   
3  GradientBoost    1.000   0.959           1.000          0.966        1.000   
4            MLP    0.885   0.869           0.908          0.893        0.863   
5       Logistic    0.837   0.829           0.882          0.875        0.797   
6       Poly_SVM    0.983   0.921           0.990          0.933        0.976   
7            SVM    0.989   0.931           0.990          0.936        0.988   

  test_recall train_accuracy test_accuracy  \
0       0.459          0.493         0.503   
1       0.928          0.987         0.932   
2       0.955          1.000         0.960   
3       0.954          1.000         0.961   
4       0.848          0.892         0.878   
5       0.789          0.851         0.844   
6       0.912          0.984         0.926   
7       0.928          0.990         0.935   

                                              params  
0  {'Dummy__random_state': 23, 'Dummy__strategy':...  
1   {'Tree__max_depth': 9, 'Tree__random_state': 23}  
2  {'Forest__max_depth': 15, 'Forest__random_stat...  
3  {'GradientBoost__max_depth': 5, 'GradientBoost...  
4  {'MLP__hidden_layer_sizes': 19, 'MLP__random_s...  
5  {'Logistic__C': 166.81005372000593, 'Logistic_...  
6  {'Poly_SVM__C': 1438.449888287663, 'Poly_SVM__...  
7  {'SVM__C': 29.76351441631318, 'SVM__gamma': 0....

In [39]:
print(results.to_latex(index=False))

\begin{tabular}{llllllllll}
\toprule
    algorithm & train\_f1 & test\_f1 & train\_precision & test\_precision & train\_recall & test\_recall & train\_accuracy & test\_accuracy &                                             params \\
\midrule
        Dummy &    0.450 &   0.470 &           0.470 &          0.482 &        0.431 &       0.459 &          0.493 &         0.503 & \{'Dummy\_\_random\_state': 23, 'Dummy\_\_strategy': ... \\
         Tree &    0.986 &   0.929 &           0.986 &          0.931 &        0.987 &       0.928 &          0.987 &         0.932 &   \{'Tree\_\_max\_depth': 9, 'Tree\_\_random\_state': 23\} \\
       Forest &    1.000 &   0.959 &           1.000 &          0.963 &        1.000 &       0.955 &          1.000 &         0.960 & \{'Forest\_\_max\_depth': 15, 'Forest\_\_random\_state... \\
GradientBoost &    1.000 &   0.959 &           1.000 &          0.966 &        1.000 &       0.954 &          1.000 &         0.961 & \{'GradientBoost\_\_max\_depth': 5, 'Gr

/var/folders/kv/34jyv69d3lq7qnz8d_2l_s4w0000gq/T/ipykernel_1359/2695231829.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results.to_latex(index=False))


**original results**

In [62]:
results

algorithm  train_f1   test_f1  train_precision  test_precision  \
0          Dummy  0.449869  0.470000         0.470351        0.481795   
1           Tree  0.986496  0.929088         0.986204        0.931148   
2         Forest  1.000000  0.958637         1.000000        0.963214   
3  GradientBoost  1.000000  0.959233         1.000000        0.965695   
4       Logistic  0.825355  0.816681         0.870500        0.866343   
5            SVM  0.995341  0.924322         0.994384        0.927474   

   train_recall  test_recall  \
0      0.431097     0.458778   
1      0.986815     0.927660   
2      1.000000     0.955176   
3      1.000000     0.953706   
4      0.784692     0.774203   
5      0.996302     0.921822   

                                              params  
0  {'Dummy__random_state': 23, 'Dummy__strategy':...  
1   {'Tree__max_depth': 9, 'Tree__random_state': 23}  
2  {'Forest__max_depth': 15, 'Forest__random_stat...  
3  {'GradientBoost__max_depth': 5, 'GradientBoost...  
4                     {'Logistic__random_state': 23}  
5  {'SVM__C': 129.1549665014884, 'SVM__gamma': 0....

In [63]:
print(results.to_latex(index=False))

\begin{tabular}{lrrrrrrl}
\toprule
    algorithm &  train\_f1 &  test\_f1 &  train\_precision &  test\_precision &  train\_recall &  test\_recall &                                             params \\
\midrule
        Dummy &  0.449869 & 0.470000 &         0.470351 &        0.481795 &      0.431097 &     0.458778 & \{'Dummy\_\_random\_state': 23, 'Dummy\_\_strategy': ... \\
         Tree &  0.986496 & 0.929088 &         0.986204 &        0.931148 &      0.986815 &     0.927660 &   \{'Tree\_\_max\_depth': 9, 'Tree\_\_random\_state': 23\} \\
       Forest &  1.000000 & 0.958637 &         1.000000 &        0.963214 &      1.000000 &     0.955176 & \{'Forest\_\_max\_depth': 15, 'Forest\_\_random\_state... \\
GradientBoost &  1.000000 & 0.959233 &         1.000000 &        0.965695 &      1.000000 &     0.953706 & \{'GradientBoost\_\_max\_depth': 5, 'GradientBoost\_... \\
     Logistic &  0.825355 & 0.816681 &         0.870500 &        0.866343 &      0.784692 &     0.774203 &             

/var/folders/kv/34jyv69d3lq7qnz8d_2l_s4w0000gq/T/ipykernel_23340/2695231829.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results.to_latex(index=False))


**TODO:** 
- Add number of bids as feature **DONE**
- Need to figure out this shitty error talk to MJ **DONE**
- evtl. Try feature selection with Tree - coul probably help to improve Tree performance **DONE**
- Control fucking randomness in Feature Selection **DONE**

Observation: Performs 2-5 percentpoints better in Logisitc regression makes kind of sense because the feature selection happend via a linear SVM.(Not quit sure, discuss with Marc)

# Second Method for Feature Selection (with feature tools)

Use Featuretools, interesting because the data could be split in two dataframes.

In [3]:
import featuretools as ft

In [38]:
features_ML = ["cv", "spread", "diffp", "rd", "rdnorm", "rdalt", "skew", "ks"]

In [39]:
df_brazil = pd.read_json(data_dir/"brazil_gaz_train.json")
df_brazil_markers = df_brazil.groupby(by="ID").head(1).reset_index()
df_brazil_markers = df_brazil_markers[features_ML+["ID"]]
df_brazil_markers.head()

cv    spread     diffp        rd     rdnorm      rdalt      skew  \
0  0.036899  0.291240  0.000569  0.014614   0.646484   0.645793  1.495001   
1  0.009570  0.049772  0.013699  1.902243   7.155963   9.493671 -2.044359   
2  0.036081  0.285391  0.001699  0.044780   1.970238   1.976048  1.585147   
3  0.034239  0.319767  0.026744  0.727912  27.349091  29.753968  1.326045   
4  0.033923  0.275998  0.000000  0.000000   0.000000   0.000000  1.446969   

         ks  ID  
0  5.020408   1  
1  3.543623   2  
2  5.121760   3  
3  5.518785   5  
4  5.572819   7

In [40]:
df_brazil_prices = df_brazil[["ID","Price"]].reset_index()
df_brazil_prices.head()

index  ID  Price
0      0   1  1.758
1      1   1  1.759
2      2   1  1.759
3      3   1  1.765
4      4   1  1.766

In [41]:
dataframes = {
    "markers": (df_brazil_markers, "ID"),
    "prices": (df_brazil_prices, "index"),
    
}

In [42]:
relationships = [
    ("markers", "ID", "prices", "ID"),
]

In [43]:
feature_matrix, features_defs = ft.dfs(
    dataframes=dataframes,
    relationships=relationships,
    target_dataframe_name="markers",
)
feature_matrix

cv    spread     diffp        rd     rdnorm      rdalt      skew  \
ID                                                                             
1     0.036899  0.291240  0.000569  0.014614   0.646484   0.645793  1.495001   
2     0.009570  0.049772  0.013699  1.902243   7.155963   9.493671 -2.044359   
3     0.036081  0.285391  0.001699  0.044780   1.970238   1.976048  1.585147   
5     0.034239  0.319767  0.026744  0.727912  27.349091  29.753968  1.326045   
7     0.033923  0.275998  0.000000  0.000000   0.000000   0.000000  1.446969   
...        ...       ...       ...       ...        ...        ...       ...   
1918  0.011411  0.046522  0.023261  2.117803  24.000000  47.000000  0.255113   
1919  0.028115  0.162829  0.023494  0.818167   9.811429  11.297160 -0.069239   
1920  0.030100  0.119076  0.000000  0.000000   0.000000   0.000000  0.185375   
1921  0.025457  0.091723  0.002184  0.096560   0.333333   0.317073 -1.049228   
1922  0.017316  0.069784  0.000000  0.000000   0.000000   0.000000  1.189888   

            ks  COUNT(prices)  MAX(prices.Price)  MEAN(prices.Price)  \
ID                                                                     
1     5.020408            332              2.270            1.857554   
2     3.543623             27              2.299            2.266074   
3     5.121760            332              2.270            1.859485   
5     5.518785            328              2.270            1.855954   
7     5.572819            328              2.270            1.856296   
...        ...            ...                ...                 ...   
1918  2.643120             49              4.499            4.431653   
1919  3.117807             69              4.999            4.709971   
1920  1.805144             47              4.699            4.431447   
1921  2.182834             15              4.999            4.840267   
1922  2.154038             49              4.599            4.362265   

      MIN(prices.Price)  SKEW(prices.Price)  STD(prices.Price)  \
ID                                                               
1                 1.758            1.495001           0.068542   
2                 2.190           -2.044359           0.021686   
3                 1.766            1.585147           0.067091   
5                 1.720            1.326045           0.063545   
7                 1.779            1.446969           0.062971   
...                 ...                 ...                ...   
1918              4.299            0.255113           0.050571   
1919              4.299           -0.069239           0.132421   
1920              4.199            0.185375           0.133387   
1921              4.579           -1.049228           0.123220   
1922              4.299            1.189888           0.075537   

      SUM(prices.Price)  
ID                       
1               616.708  
2                61.184  
3               617.349  
5               608.753  
7               608.865  
...                 ...  
1918            217.151  
1919            324.988  
1920            208.278  
1921             72.604  
1922            213.751  

[1438 rows x 15 columns]